In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
import seaborn as sns


sns.set(style="whitegrid")
palette = sns.color_palette("husl", 8)
ITALIAN_COLOR = sns.color_palette("husl", 8)[0]
ENGLISH_COLOR = sns.color_palette("husl", 8)[3]

## Imports

In [5]:
!python -m spacy download it_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 24.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('it_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [6]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 14.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [7]:
!pip install transformers torch pandas scikit-learn

  Attempting uninstall: nvidia-cusparse-cu12
    Found existing installation: nvidia-cusparse-cu12 12.5.1.3
    Uninstalling nvidia-cusparse-cu12-12.5.1.3:
      Successfully uninstalled nvidia-cusparse-cu12-12.5.1.3
  Attempting uninstall: nvidia-cudnn-cu12
    Found existing installation: nvidia-cudnn-cu12 9.3.0.75
    Uninstalling nvidia-cudnn-cu12-9.3.0.75:
      Successfully uninstalled nvidia-cudnn-cu12-9.3.0.75
  Attempting uninstall: nvidia-cusolver-cu12
    Found existing installation: nvidia-cusolver-cu12 11.6.3.83
    Uninstalling nvidia-cusolver-cu12-11.6.3.83:
      Successfully uninstalled nvidia-cusolver-cu12-11.6.3.83


In [45]:
import pandas as pd
import re
import spacy
import sys

import torch
import torch.nn as nn
from transformers import AutoModel, AutoTokenizer
from transformers import AutoTokenizer, AutoModelForMaskedLM
import umap

from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import mean_squared_error, silhouette_score, adjusted_rand_score, mutual_info_score, jaccard_score, pairwise_distances
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import ParameterGrid
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import trustworthiness
from scipy.stats import spearmanr

from sklearn.cluster import DBSCAN, SpectralClustering
from scipy.sparse import csr_matrix
from scipy.cluster.hierarchy import linkage, fcluster
from scipy.spatial.distance import squareform
from concurrent.futures import ThreadPoolExecutor

from tqdm import tqdm
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.colors import to_hex

from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.spatial.distance import cdist, pdist

from hdbscan import HDBSCAN
import nltk
from nltk.util import bigrams

import pickle

from transformers import AutoTokenizer, AutoModelForMaskedLM
from sklearn.metrics import calinski_harabasz_score, davies_bouldin_score

from sklearn.preprocessing import normalize

from sklearn.cluster import DBSCAN, SpectralClustering
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
from sklearn.model_selection import ParameterGrid
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import pdist, squareform
from scipy.sparse.csgraph import laplacian
import hdbscan
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import combinations
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from scipy.spatial import ConvexHull
import matplotlib.patches as mpatches
from matplotlib.patches import Ellipse

import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import pairwise_distances
from scipy.spatial.distance import cdist


from transformers import AutoTokenizer, AutoModelForMaskedLM
from sklearn.metrics import calinski_harabasz_score
from sklearn.metrics import silhouette_score, davies_bouldin_score

from sklearn.cluster import DBSCAN, SpectralClustering
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
from sklearn.model_selection import ParameterGrid
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import pdist, squareform
from scipy.sparse.csgraph import laplacian
import hdbscan
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import combinations


## Verbs lists

### Italian

In [10]:
italian_verbs = ['toccare', 'sfiorare', 'colpire', 'urtare', 'sbattere', 'tamponare',
                'scontrarsi', 'spingere', 'spintonare', 'schiaffeggiare', 'calciare',
                'leccare', 'baciare', 'frustare', 'punzecchiare', 'pungere', 'schiacciare',
                'premere', 'cliccare', 'appoggiare', 'poggiare'
                 ]

In [9]:
italian_to_russian = {
    "toccare": "тронуть",
    "sfiorare": "коснуться",
    "colpire": "ударить",
    "urtare": "удариться",
    "sbattere": "удариться",
    "tamponare": "врезаться",
    "scontrarsi": "столкнуться",
    "spingere": "толкнуть",
    "spintonare": "толкнуть",
    "schiaffeggiare": "шлёпнуть",
    "calciare": "пнуть",
    "leccare": "лизнуть",
    "baciare": "поцеловать",
    "frustare": "хлестнуть",
    "punzecchiare": "кольнуть",
    "pungere": "кольнуть",
    "schiacciare": "нажать",
    "premere": "нажать",
    "cliccare": "кликнуть",
    "appoggiare": "приложить",
    "poggiare": "приложить"
}

### English

In [11]:
english_verbs = ['touch', 'hit', 'strike', 'beat', 'bang', 'bash', 'punch', 'whack',
                 'kick', 'smack', 'slap', 'swat', 'pat', 'tap', 'push', 'shove',
                 'crash', 'bump', 'collide', 'graze', 'nick'
                 ]

In [12]:
english_to_russian = {
    "touch": "тронуть",
    "hit": "ударить",
    "strike": "ударить",
    "beat": "ударить",
    "bang": "ударить",
    "bash": "разбить",
    "punch": "ударить",
    "whack": "шлёпнуть",
    "kick": "пнуть",
    "smack": "шлёпнуть",
    "slap": "шлёпнуть",
    "swat": "поймать",
    "pat": "погладить",
    "tap": "постучать",
    "push": "толкнуть",
    "shove": "толкнуть",
    "crash": "разбиться",
    "bump": "столкнуться",
    "collide": "столкнуться",
    "graze": "коснуться",
    "nick": "царапнуть"
}

## Italian Tweets

In [ ]:
italian_df = pd.read_csv("/content/drive/MyDrive/Диплом бакалавриат'25/final_italian_7to23.csv")

In [ ]:
italian_df['found_verbs'].value_counts()

,count
found_verbs,
toccare (тронуть),3967
colpire (ударить),3925
spingere (толкнуть),2819
tamponare (врезаться),651
cliccare (кликнуть),499
appoggiare (приложить),462
sbattere (удариться),460
sfiorare (коснуться),394
premere (нажать),248


In [ ]:
def sample_verbs(df: pd.DataFrame, threshold: int = 460) -> pd.DataFrame:
    """
    Извлекает случайные контексты для глаголов, которые встречаются более заданного количества раз.

    :param df: DataFrame с данными
    :param threshold: Минимальное количество вхождений для выбора глагола
    :return: DataFrame с отобранными контекстами
    """

    verb_counts = df['found_verbs'].value_counts()

    common_verbs = verb_counts[verb_counts >= threshold].index

    sampled_contexts_list = []

    for verb in common_verbs:
        contexts = df[df['found_verbs'] == verb]

        sampled_contexts = contexts.sample(n=min(threshold, len(contexts)), random_state=42)
        sampled_contexts_list.append(sampled_contexts)

    return pd.concat(sampled_contexts_list, ignore_index=True)


In [ ]:
italian_contexts = sample_verbs(italian_df)

In [ ]:
def enumerate_verbs(df):
    df = df.sort_values(by='found_verbs').reset_index(drop=True)

    df['context_id'] = df.groupby('found_verbs').cumcount() + 1

    df['context_id'] = df['found_verbs'] + '_' + df['context_id'].astype(str)

    return df

italian_contexts = enumerate_verbs(italian_contexts)

In [ ]:
italian_contexts = italian_contexts[['id', 'context_id', 'text',  'context', 'found_verbs', 'spacy_word_count']]
italian_contexts['found_verbs'].value_counts()

,count
found_verbs,
appoggiare (приложить),460
cliccare (кликнуть),460
colpire (ударить),460
sbattere (удариться),460
spingere (толкнуть),460
tamponare (врезаться),460
toccare (тронуть),460


In [ ]:
italian_contexts.to_csv("/content/drive/MyDrive/Диплом бакалавриат'25/italian_contexts.csv")

### UmBERTo

In [ ]:
model_name = "Musixmatch/umberto-commoncrawl-cased-v1"
model = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

#### Предобработка текста

In [ ]:
italian_df = pd.read_csv("/content/drive/MyDrive/Диплом бакалавриат'25/italian_contexts.csv")
italian_df = italian_df[['id', 'context_id', 'text', 'context', 'found_verbs', 'spacy_word_count']]

In [ ]:
def preprocess_tweet(tweet):
    # Удаление URL
    tweet = re.sub(r'http\S+', '', tweet)
    tweet = re.sub(r'@link', '', tweet)
    # Замена упоминаний
    tweet = re.sub(r'@\w+', '@user', tweet)
    # Обработка хэштегов
    tweet = re.sub(r'#(\w+)', lambda x: x.group(1), tweet)
    # Удаление линих пробелов
    tweet = re.sub(r"\s+", ' ', tweet)

    return tweet.strip()

In [ ]:
italian_df['cleaned_text'] = italian_df['context'].apply(preprocess_tweet)

In [ ]:
italian_df.to_csv("/content/drive/MyDrive/Диплом бакалавриат'25/italian_contexts.csv")

#### Создание эмбеддингов

In [12]:
italian_df = pd.read_csv("/content/drive/MyDrive/Диплом бакалавриат'25/italian_contexts.csv")
italian_df = italian_df[['id', 'context_id', 'text',  'context', 'cleaned_text', 'found_verbs', 'spacy_word_count']]

In [13]:
nlp = spacy.load("it_core_news_sm")
target_verbs = np.unique([verb.split()[0] for verb in italian_df['found_verbs'].tolist()])

def extract_verbs(df):
    """Извлечение целевых глаголов из оригинального текста"""
    verb_records = []

    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Extracting verbs"):
        doc = nlp(row['context'])
        verbs = []

        for token in doc:
            if token.pos_ == "VERB":
                for target_verb in target_verbs:
                    # Проверка частичного/полного совпадения
                    if target_verb in token.lemma_ or token.lemma_ in target_verb:
                        verbs.append({
                            'raw_text': token.text,  # Оригинальная словоформа
                            'pattern': target_verb
                        })
                        break

        verb_records.append(verbs)

    df['verbs_info'] = verb_records
    return df

In [ ]:
italian_df = extract_verbs(italian_df)

In [19]:
def create_masked_text(df):
    """Создает отдельные записи для каждого целевого глагола"""

    new_rows = []

    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Processing rows"):
        text = row['cleaned_text']
        verbs = row['verbs_info']

        # Для каждого глагола создаем отдельную маску
        for i, verb_info in enumerate(verbs):
            new_row = row.copy()
            target_verb = verb_info['raw_text']

            # Точная замена только конкретного глагола
            masked_text = re.sub(
                r'\b{}\b'.format(re.escape(target_verb)),
                '<mask>',
                text,
                count=1  # Заменяем только первое вхождение
            )

            # Проверка успешности замены
            if masked_text == text:
                print(idx)
                continue  # Пропускаем неудачные замены

            # Обновляем информацию о глаголах
            new_verbs = [v for j, v in enumerate(verbs) if j != i]
            new_row['verbs_info'] = verbs
            new_row['cleaned_text'] = masked_text
            new_row['masked_verb'] = target_verb  # Добавляем новое поле

            new_rows.append(new_row)

    # Создаем новый DataFrame
    return pd.DataFrame(new_rows).reset_index(drop=True)

In [ ]:
it_featured_df = create_masked_text(italian_df)

In [ ]:
it_featured_df = it_featured_df[it_featured_df['masked_verb'].apply(len) > 0]
len(it_featured_df)

##### Эксперимент с выбором слоев и методов усреднения

In [ ]:
class UmBERToEmbeddingExperiment:
    """
    Класс для экспериментов с различными методами извлечения эмбеддингов из UmBERTo
    """

    def __init__(self, model_name="Musixmatch/umberto-commoncrawl-cased-v1"):
        """Инициализация модели и токенизатора"""
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForMaskedLM.from_pretrained(model_name, output_hidden_states=True)
        self.model.eval()

        # Загружаем spaCy для анализа POS-тегов
        try:
            self.nlp = spacy.load("it_core_news_sm")
        except OSError:
            print("Модель it_core_news_sm не найдена.")
            self.nlp = None

        # Определяем методы извлечения эмбеддингов
        self.extraction_methods = {
            # Стандартные методы
            'last_4_avg': self._extract_last_4_average,
            'last_2_avg': self._extract_last_2_average,
            'weighted_top_4': self._extract_weighted_top_4,

            # Средние слои (избегаем нижние)
            'middle_layers_avg': self._extract_middle_layers,
            'attention_weighted_middle': self._extract_attention_weighted_middle,

            # Дополнительные методы
            'exponential_decay': self._extract_exponential_decay,
            'semantic_attention': self._extract_semantic_attention,
            'layer_dropout_ensemble': self._extract_layer_dropout_ensemble,
            'dynamic_layer_selection': self._extract_dynamic_layer_selection,
            'contrastive_pooling': self._extract_contrastive_pooling
        }

    def _get_token_embeddings(self, text, return_attention=False):
        """Базовая функция для получения скрытых состояний всех слоев"""
        inputs = self.tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)

        with torch.no_grad():
            if return_attention:
                outputs = self.model(**inputs, output_attentions=True)
                return outputs.hidden_states, outputs.attentions, inputs
            else:
                outputs = self.model(**inputs)
                return outputs.hidden_states, None, inputs

    # ========== СТАНДАРТНЫЕ МЕТОДЫ ==========

    def _extract_last_4_average(self, text):
        """Усреднение последних 4 слоев"""
        hidden_states, _, inputs = self._get_token_embeddings(text)

        # Берем последние 4 слоя
        last_4 = torch.stack(hidden_states[-4:])  # [4, batch, seq_len, hidden_dim]
        averaged = torch.mean(last_4, dim=0)  # [batch, seq_len, hidden_dim]

        # Усредняем по всем токенам (исключая специальные)
        mask = inputs['attention_mask'].bool()
        pooled = self._mean_pooling(averaged, mask)

        return pooled.squeeze().numpy()

    def _extract_last_2_average(self, text):
        """Усреднение последних 2 слоев"""
        hidden_states, _, inputs = self._get_token_embeddings(text)

        last_2 = torch.stack(hidden_states[-2:])
        averaged = torch.mean(last_2, dim=0)

        mask = inputs['attention_mask'].bool()
        pooled = self._mean_pooling(averaged, mask)

        return pooled.squeeze().numpy()

    def _extract_weighted_top_4(self, text):
        """Взвешенное усреднение топ-4 слоев с убывающими весами"""
        hidden_states, _, inputs = self._get_token_embeddings(text)

        # Веса для последних 4 слоев: [0.4, 0.3, 0.2, 0.1]
        weights = torch.tensor([0.1, 0.2, 0.3, 0.4]).unsqueeze(1).unsqueeze(2)

        last_4 = torch.stack(hidden_states[-4:])
        weighted = torch.sum(last_4 * weights, dim=0)

        mask = inputs['attention_mask'].bool()
        pooled = self._mean_pooling(weighted, mask)

        return pooled.squeeze().numpy()

    # ========== СРЕДНИЕ СЛОИ ==========

    def _extract_middle_layers(self, text):
        """Усреднение средних слоев (слои 6-9 из 12)"""
        hidden_states, _, inputs = self._get_token_embeddings(text)

        # Берем средние слои (избегаем первые 5 и последние 3)
        middle_layers = torch.stack(hidden_states[6:10])  # слои 6,7,8,9
        averaged = torch.mean(middle_layers, dim=0)

        mask = inputs['attention_mask'].bool()
        pooled = self._mean_pooling(averaged, mask)

        return pooled.squeeze().numpy()

    def _extract_attention_weighted_middle(self, text):
        """Взвешивание средних слоев на основе attention weights"""
        hidden_states, attentions, inputs = self._get_token_embeddings(text, return_attention=True)

        # Берем attention веса средних слоев
        middle_attentions = attentions[6:10]  # слои 6-9
        middle_states = torch.stack(hidden_states[6:10])

        # Усредняем attention по головам и используем как веса
        attention_weights = torch.stack([att.mean(dim=1) for att in middle_attentions])  # [4, batch, seq_len, seq_len]
        attention_weights = attention_weights.mean(dim=-1)  # [4, batch, seq_len]
        attention_weights = torch.softmax(attention_weights.mean(dim=-1), dim=0)  # [4, batch]

        # Применяем веса к состояниям
        weighted_states = torch.sum(middle_states * attention_weights.unsqueeze(-1).unsqueeze(-1), dim=0)

        mask = inputs['attention_mask'].bool()
        pooled = self._mean_pooling(weighted_states, mask)

        return pooled.squeeze().numpy()

    # ========== ДОПОЛНИТЕЛЬНЫЕ МЕТОДЫ ==========

    def _extract_exponential_decay(self, text):
        """Экспоненциальное затухание весов от последних к средним слоям"""
        hidden_states, _, inputs = self._get_token_embeddings(text)

        # Создаем экспоненциально убывающие веса
        num_layers = len(hidden_states)
        decay_factor = 0.8
        weights = torch.tensor([decay_factor ** (num_layers - i - 1) for i in range(num_layers)])
        weights = weights / weights.sum()  # нормализуем

        # Исключаем первые 4 слоя (слишком низкоуровневые)
        relevant_states = torch.stack(hidden_states[4:])
        relevant_weights = weights[4:] / weights[4:].sum()

        weighted = torch.sum(relevant_states * relevant_weights.unsqueeze(1).unsqueeze(2), dim=0)

        mask = inputs['attention_mask'].bool()
        pooled = self._mean_pooling(weighted, mask)

        return pooled.squeeze().numpy()

    def _extract_semantic_attention(self, text):
        """Адаптивное взвешивание на основе семантической важности токенов"""
        hidden_states, attentions, inputs = self._get_token_embeddings(text, return_attention=True)

        # Берем верхние слои (9-12)
        top_layers = torch.stack(hidden_states[-4:])
        top_attentions = attentions[-4:]

        # Вычисляем "семантическую важность" как дисперсию attention весов
        semantic_importance = []
        for att in top_attentions:
            # att: [batch, num_heads, seq_len, seq_len]
            att_variance = torch.var(att.mean(dim=1), dim=-1)  # [batch, seq_len]
            semantic_importance.append(att_variance.mean(dim=-1))  # [batch]

        semantic_weights = torch.stack(semantic_importance)  # [4, batch]
        semantic_weights = torch.softmax(semantic_weights, dim=0)

        weighted = torch.sum(top_layers * semantic_weights.unsqueeze(-1).unsqueeze(-1), dim=0)

        mask = inputs['attention_mask'].bool()
        pooled = self._mean_pooling(weighted, mask)

        return pooled.squeeze().numpy()

    def _extract_layer_dropout_ensemble(self, text):
        """Ансамбль с случайным исключением слоев"""
        hidden_states, _, inputs = self._get_token_embeddings(text)

        # Создаем несколько ансамблей с разными комбинациями слоев
        ensembles = []
        relevant_layers = list(range(6, len(hidden_states)))  # средние и верхние слои

        for _ in range(5):  # 5 разных ансамблей
            # Случайно выбираем 60-80% слоев
            num_select = np.random.randint(int(len(relevant_layers) * 0.6), int(len(relevant_layers) * 0.8) + 1)
            selected_indices = np.random.choice(relevant_layers, num_select, replace=False)

            selected_states = torch.stack([hidden_states[i] for i in selected_indices])
            ensemble_avg = torch.mean(selected_states, dim=0)
            ensembles.append(ensemble_avg)

        # Усредняем все ансамбли
        final_ensemble = torch.mean(torch.stack(ensembles), dim=0)

        mask = inputs['attention_mask'].bool()
        pooled = self._mean_pooling(final_ensemble, mask)

        return pooled.squeeze().numpy()

    def _extract_dynamic_layer_selection(self, text):
        """Динамический выбор слоев на основе косинусного сходства"""
        hidden_states, _, inputs = self._get_token_embeddings(text)

        # Вычисляем попарное косинусное сходство между слоями
        layer_similarities = []
        for i in range(6, len(hidden_states) - 1):  # средние и верхние слои
            state1 = hidden_states[i].mean(dim=1)  # [batch, hidden_dim]
            state2 = hidden_states[i + 1].mean(dim=1)

            similarity = torch.cosine_similarity(state1, state2, dim=-1)
            layer_similarities.append(similarity.item())

        # Выбираем слои с наименьшим сходством (наиболее информативные переходы)
        similarity_scores = np.array(layer_similarities)
        diverse_indices = np.argsort(similarity_scores)[:4]  # топ-4 наиболее различающихся
        diverse_indices = [i + 6 for i in diverse_indices]  # корректируем индексы

        selected_states = torch.stack([hidden_states[i] for i in diverse_indices])
        averaged = torch.mean(selected_states, dim=0)

        mask = inputs['attention_mask'].bool()
        pooled = self._mean_pooling(averaged, mask)

        return pooled.squeeze().numpy()

    def _extract_contrastive_pooling(self, text):
        """Контрастивное объединение: подчеркиваем различия между слоями"""
        hidden_states, _, inputs = self._get_token_embeddings(text)

        # Берем верхние слои
        top_layers = hidden_states[-4:]

        # Вычисляем "контрастивные" веса
        contrasts = []
        for i, layer in enumerate(top_layers):
            if i == 0:
                contrasts.append(layer)
            else:
                # Подчеркиваем различия с предыдущим слоем
                contrast = layer - 0.3 * top_layers[i-1]
                contrasts.append(contrast)

        # Комбинируем контрастивные представления
        final_contrast = torch.mean(torch.stack(contrasts), dim=0)

        mask = inputs['attention_mask'].bool()
        pooled = self._mean_pooling(final_contrast, mask)

        return pooled.squeeze().numpy()

    def _mean_pooling(self, token_embeddings, attention_mask):
        """Усреднение токенов с учетом маски внимания"""
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        pooled = torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

        # Убеждаемся, что возвращаем 2D тензор [batch_size, hidden_dim]
        if pooled.dim() == 1:
            pooled = pooled.unsqueeze(0)
        elif pooled.dim() > 2:
            pooled = pooled.view(pooled.size(0), -1)

        return pooled

    def extract_embeddings_for_method(self, texts, method_name):
        """Извлечение эмбеддингов для всех текстов с выбранным методом"""
        method = self.extraction_methods[method_name]
        embeddings = []

        print(f"Извлечение эмбеддингов методом '{method_name}'...")
        for text in tqdm(texts, desc=f"Обработка {method_name}",
                        leave=True, ncols=100, file=sys.stdout):
            try:
                embedding = method(text)

                # Проверяем и исправляем размерность
                if isinstance(embedding, np.ndarray):
                    if embedding.ndim == 1:
                        embeddings.append(embedding)
                    elif embedding.ndim == 2:
                        # Если 2D, берем первую строку (batch_size=1)
                        embeddings.append(embedding[0])
                    else:
                        # Если больше 2D, сглаживаем
                        embeddings.append(embedding.flatten()[:768])  # обрезаем до нужной размерности
                else:
                    # Если не numpy array, конвертируем
                    embedding = np.array(embedding)
                    if embedding.ndim > 1:
                        embedding = embedding.flatten()[:768]
                    embeddings.append(embedding)

            except Exception as e:
                print(f"Ошибка для текста: {text[:50]}... - {e}")
                # Добавляем нулевой вектор в случае ошибки
                embeddings.append(np.zeros(768))  # размерность UmBERTo

        # Финальная проверка размерности
        embeddings_array = np.array(embeddings)

        # Если все еще есть проблемы с размерностью, приводим к правильной форме
        if embeddings_array.ndim != 2:
            print(f"Исправление размерности массива: {embeddings_array.shape} -> (?, 768)")
            embeddings_list = []
            for emb in embeddings:
                if isinstance(emb, np.ndarray):
                    if emb.ndim == 0:
                        embeddings_list.append(np.zeros(768))
                    elif emb.ndim == 1:
                        if len(emb) >= 768:
                            embeddings_list.append(emb[:768])
                        else:
                            padded = np.zeros(768)
                            padded[:len(emb)] = emb
                            embeddings_list.append(padded)
                    else:
                        flattened = emb.flatten()
                        if len(flattened) >= 768:
                            embeddings_list.append(flattened[:768])
                        else:
                            padded = np.zeros(768)
                            padded[:len(flattened)] = flattened
                            embeddings_list.append(padded)
                else:
                    embeddings_list.append(np.zeros(768))
            embeddings_array = np.array(embeddings_list)

        print(f"Итоговая размерность эмбеддингов: {embeddings_array.shape}")
        return embeddings_array

    def evaluate_clustering_quality(self, embeddings, labels=None):
        """Оценка качества кластеризации"""

        # Проверяем размерность входных данных
        print(f"Проверка размерности эмбеддингов: {embeddings.shape}")

        if embeddings.ndim != 2:
            print(f"Неправильная размерность эмбеддингов: {embeddings.shape}")
            return {
                'silhouette_score': -1,
                'davies_bouldin_score': float('inf'),
                'calinski_harabasz_score': 0,
                'trustworthiness': 0,
                'n_clusters': 0,
                'noise_ratio': 1.0,
                'umap_embeddings': np.zeros((len(embeddings), 2)),
                'cluster_labels': np.full(len(embeddings), -1)
            }

        # Проверяем на NaN и inf
        if np.any(np.isnan(embeddings)) or np.any(np.isinf(embeddings)):
            print("Найдены NaN или inf значения в эмбеддингах")
            embeddings = np.nan_to_num(embeddings, nan=0.0, posinf=1.0, neginf=-1.0)

        try:
            # Применяем UMAP для снижения размерности
            umap_reducer = umap.UMAP(
                n_components=2,
                n_neighbors=min(20, len(embeddings)-1),  # учитываем размер данных
                min_dist=0.1,
                metric='cosine',
                random_state=42
            )

            umap_embeddings = umap_reducer.fit_transform(embeddings)

            # Кластеризация с HDBSCAN
            min_cluster_size = max(3, min(5, len(embeddings)//10))  # адаптивный размер
            clusterer = HDBSCAN(min_cluster_size=min_cluster_size, metric='euclidean')
            cluster_labels = clusterer.fit_predict(umap_embeddings)

        except Exception as e:
            print(f"Ошибка при UMAP/кластеризации: {e}")
            return {
                'silhouette_score': -1,
                'davies_bouldin_score': float('inf'),
                'calinski_harabasz_score': 0,
                'trustworthiness': 0,
                'n_clusters': 0,
                'noise_ratio': 1.0,
                'umap_embeddings': np.zeros((len(embeddings), 2)),
                'cluster_labels': np.full(len(embeddings), -1)
            }

        # Исключаем шум (-1 метки) для вычисления метрик
        valid_mask = cluster_labels != -1
        if valid_mask.sum() < 2:
            return {
                'silhouette_score': -1,
                'davies_bouldin_score': float('inf'),
                'calinski_harabasz_score': 0,
                'trustworthiness': 0,
                'n_clusters': 0,
                'noise_ratio': 1.0,
                'umap_embeddings': umap_embeddings,
                'cluster_labels': cluster_labels
            }

        valid_embeddings = umap_embeddings[valid_mask]
        valid_labels = cluster_labels[valid_mask]

        # Вычисляем метрики
        try:
            silhouette = silhouette_score(valid_embeddings, valid_labels)
            davies_bouldin = davies_bouldin_score(valid_embeddings, valid_labels)
            calinski_harabasz = calinski_harabasz_score(valid_embeddings, valid_labels)
        except Exception as e:
            print(f"Ошибка при вычислении метрик: {e}")
            silhouette = -1
            davies_bouldin = float('inf')
            calinski_harabasz = 0

        # Вычисляем trustworthiness
        try:
            from sklearn.manifold import trustworthiness as sklearn_trustworthiness
            trustworthiness = sklearn_trustworthiness(embeddings, umap_embeddings, n_neighbors=min(10, len(embeddings)-1))
        except Exception as e:
            print(f"Ошибка при вычислении trustworthiness: {e}")
            trustworthiness = 0

        return {
            'silhouette_score': silhouette,
            'davies_bouldin_score': davies_bouldin,
            'calinski_harabasz_score': calinski_harabasz,
            'trustworthiness': trustworthiness,
            'n_clusters': len(np.unique(valid_labels)),
            'noise_ratio': (cluster_labels == -1).mean(),
            'umap_embeddings': umap_embeddings,
            'cluster_labels': cluster_labels
        }

    def _calculate_trustworthiness(self, X_original, X_embedded, k=10):
        """Вычисление trustworthiness score"""
        from sklearn.neighbors import NearestNeighbors

        # Находим ближайших соседей в исходном и проецированном пространстве
        nn_original = NearestNeighbors(n_neighbors=k+1, metric='cosine').fit(X_original)
        nn_embedded = NearestNeighbors(n_neighbors=k+1, metric='euclidean').fit(X_embedded)

        _, indices_original = nn_original.kneighbors(X_original)
        _, indices_embedded = nn_embedded.kneighbors(X_embedded)

        n = X_original.shape[0]
        trustworthiness = 0

        for i in range(n):
            # Исключаем сам элемент (первый в списке соседей)
            neighbors_original = set(indices_original[i][1:])
            neighbors_embedded = set(indices_embedded[i][1:])

            # Находим "интрудеров" - точки, которые близки в проекции, но далеки в оригинале
            intruders = neighbors_embedded - neighbors_original

            rank_sum = 0
            for intruder in intruders:
                # Находим ранг интрудера в исходном пространстве
                rank = np.where(indices_original[i] == intruder)[0]
                if len(rank) > 0:
                    rank_sum += max(0, rank[0] - k)

            trustworthiness += rank_sum

        # Нормализация
        max_sum = k * (2 * n - 3 * k - 1) / 2
        trustworthiness = 1 - (2 * trustworthiness) / (n * max_sum) if max_sum > 0 else 1

        return max(0, trustworthiness)

    def run_comprehensive_experiment(self, df, text_column='cleaned_text', verb_column='masked_verb'):
        """Запуск полного эксперимента со всеми методами"""
        texts = df[text_column].tolist()
        verbs = df[verb_column].tolist() if verb_column in df.columns else None

        print(f"Начинаем эксперимент с {len(texts)} текстами и {len(self.extraction_methods)} методами")
        print(f"Методы: {list(self.extraction_methods.keys())}")

        results = {}
        embeddings_cache = {}

        # Извлекаем эмбеддинги для каждого метода
        for method_name in tqdm(self.extraction_methods.keys(),
                               desc="Обработка методов", leave=True, ncols=100):
            embeddings = self.extract_embeddings_for_method(texts, method_name)
            embeddings_cache[method_name] = embeddings

            # Оцениваем качество кластеризации
            print(f"Оценка качества для метода '{method_name}'...")
            quality_metrics = self.evaluate_clustering_quality(embeddings, verbs)

            results[method_name] = {
                'embeddings': embeddings,
                'metrics': quality_metrics
            }

            print(f"{method_name}: Silhouette={quality_metrics['silhouette_score']:.3f}, "
                  f"Clusters={quality_metrics['n_clusters']}, "
                  f"Trustworthiness={quality_metrics['trustworthiness']:.3f}")

        return results

    def plot_comparison_results(self, results, save_path=None):
          """
          Визуализация результатов сравнения методов извлечения эмбеддингов.

          Args:
              results (dict): Результаты выполнения экспериментов
              save_path (str, optional): Путь для сохранения графиков

          Returns:
              list: Ранжированный список методов по композитному рейтингу
          """
          # Настройка стиля графиков
          sns.set(style="whitegrid")
          palette = sns.color_palette("husl", 8)
          plt.figure(figsize=(14, 10))


    def plot_comparison_results(self, results, save_path=None):
          """Визуализация результатов сравнения методов"""
          # Подготавливаем данные для сравнения
          methods = list(results.keys())
          metrics_data = {
              'Method': [],
              'Silhouette Score': [],
              'Davies-Bouldin Score': [],
              'Calinski-Harabasz Score': [],
              'Trustworthiness': [],
              'N Clusters': [],
              'Noise Ratio': []
          }

          for method in methods:
              metrics = results[method]['metrics']
              metrics_data['Method'].append(method)
              metrics_data['Silhouette Score'].append(metrics['silhouette_score'])
              metrics_data['Davies-Bouldin Score'].append(metrics['davies_bouldin_score'])
              metrics_data['Calinski-Harabasz Score'].append(metrics['calinski_harabasz_score'])
              metrics_data['Trustworthiness'].append(metrics['trustworthiness'])
              metrics_data['N Clusters'].append(metrics['n_clusters'])
              metrics_data['Noise Ratio'].append(metrics['noise_ratio'])

          # Создаем большую фигуру для всех графиков
          fig, axes = plt.subplots(2, 3, figsize=(18, 12))
          fig.suptitle('Сравнение методов извлечения эмбеддингов UmBERTo', fontsize=16, fontweight='bold')

          # График 1: Silhouette Score
          ax1 = axes[0, 0]
          bars1 = ax1.bar(range(len(methods)), metrics_data['Silhouette Score'],
                        color=palette[:len(methods)])
          ax1.set_title('Silhouette Score (выше = лучше)')
          ax1.set_xlabel('Методы')
          ax1.set_ylabel('Score')
          ax1.set_xticks(range(len(methods)))
          ax1.set_xticklabels(methods, rotation=45, ha='right')
          ax1.grid(True, alpha=0.3)

          # График 2: Davies-Bouldin Score
          ax2 = axes[0, 1]
          bars2 = ax2.bar(range(len(methods)), metrics_data['Davies-Bouldin Score'],
                        color=palette[:len(methods)])
          ax2.set_title('Davies-Bouldin Score (ниже = лучше)')
          ax2.set_xlabel('Методы')
          ax2.set_ylabel('Score')
          ax2.set_xticks(range(len(methods)))
          ax2.set_xticklabels(methods, rotation=45, ha='right')
          ax2.grid(True, alpha=0.3)

          # График 3: Trustworthiness
          ax3 = axes[0, 2]
          bars3 = ax3.bar(range(len(methods)), metrics_data['Trustworthiness'],
                        color=palette[:len(methods)])
          ax3.set_title('Trustworthiness (выше = лучше)')
          ax3.set_xlabel('Методы')
          ax3.set_ylabel('Score')
          ax3.set_xticks(range(len(methods)))
          ax3.set_xticklabels(methods, rotation=45, ha='right')
          ax3.grid(True, alpha=0.3)

          # График 4: Количество кластеров
          ax4 = axes[1, 0]
          bars4 = ax4.bar(range(len(methods)), metrics_data['N Clusters'],
                        color=palette[:len(methods)])
          ax4.set_title('Количество кластеров')
          ax4.set_xlabel('Методы')
          ax4.set_ylabel('Количество')
          ax4.set_xticks(range(len(methods)))
          ax4.set_xticklabels(methods, rotation=45, ha='right')
          ax4.grid(True, alpha=0.3)

          # График 5: Доля шума
          ax5 = axes[1, 1]
          bars5 = ax5.bar(range(len(methods)), metrics_data['Noise Ratio'],
                        color=palette[:len(methods)])
          ax5.set_title('Доля шума (ниже = лучше)')
          ax5.set_xlabel('Методы')
          ax5.set_ylabel('Доля')
          ax5.set_xticks(range(len(methods)))
          ax5.set_xticklabels(methods, rotation=45, ha='right')
          ax5.grid(True, alpha=0.3)

          # График 6: Комбинированный рейтинг
          ax6 = axes[1, 2]
          # Нормализуем метрики и создаем составной рейтинг
          normalized_silhouette = np.array(metrics_data['Silhouette Score'])
          normalized_trustworthiness = np.array(metrics_data['Trustworthiness'])
          normalized_db = 1 / (1 + np.array(metrics_data['Davies-Bouldin Score']))  # инвертируем
          normalized_noise = 1 - np.array(metrics_data['Noise Ratio'])  # инвертируем

          composite_score = (normalized_silhouette + normalized_trustworthiness +
                            normalized_db + normalized_noise) / 4

          bars6 = ax6.bar(range(len(methods)), composite_score,
                        color=palette[:len(methods)])
          ax6.set_title('Композитный рейтинг (выше = лучше)')
          ax6.set_xlabel('Методы')
          ax6.set_ylabel('Score')
          ax6.set_xticks(range(len(methods)))
          ax6.set_xticklabels(methods, rotation=45, ha='right')
          ax6.grid(True, alpha=0.3)

          plt.tight_layout()

          if save_path:
              plt.savefig(save_path, dpi=300, bbox_inches='tight')

          plt.show()

          # Возвращаем рейтинг методов
          ranking = sorted(zip(methods, composite_score), key=lambda x: x[1], reverse=True)
          return ranking

In [ ]:
experiment = UmBERToEmbeddingExperiment()

# Запуск полного эксперимента
results = experiment.run_comprehensive_experiment(
    it_featured_df[:200],
    text_column='cleaned_text',    # колонка с маскированными текстами
    verb_column='masked_verb'      # колонка с замаскированными глаголами
)

# Визуализация результатов
ranking = experiment.plot_comparison_results(results)

# Вывод лучших методов
print("ТОП-3 метода:")
for i, (method, score) in enumerate(ranking[:3], 1):
    print(f"{i}. {method}: {score:.4f}")

#### Cоздание эмбеддингов с помощью лучшего метода агрегации

In [ ]:
def generate_weighted_top4_embeddings(
    texts,
    model,
    tokenizer,
    layer_weights=[0.1, 0.2, 0.3, 0.4],
    max_length=512,
    device="cpu",
    batch_size=16,
    show_progress=True
):
    """
    Генерирует эмбеддинги с прогресс-баром и пакетной обработкой

    Параметры:
        batch_size (int): Размер пакета для обработки
        show_progress (bool): Показывать прогресс-бар
    """
    # Валидация входных данных
    if not texts:
        return np.array([])

    if len(layer_weights) != 4:
        raise ValueError("layer_weights должен содержать 4 элемента")

    # Подготовка прогресс-бара
    total_batches = (len(texts) + batch_size - 1) // batch_size
    pbar = tqdm(total=total_batches, desc="Generating embeddings",
                disable=not show_progress)

    all_embeddings = []

    # Обработка батчами
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i+batch_size]

        # Токенизация батча
        inputs = tokenizer(
            batch_texts,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=max_length
        ).to(device)

        # Получение эмбеддингов
        with torch.no_grad():
            outputs = model(**inputs)
            hidden_states = outputs.hidden_states

        # Взвешенное объединение слоев
        last_4 = torch.stack(hidden_states[-4:])
        weights = torch.tensor(layer_weights, device=device).view(4, 1, 1, 1)
        weighted = (last_4 * weights).sum(dim=0)

        # Пулинг с маской
        mask = inputs['attention_mask'].unsqueeze(-1).float()
        summed = torch.sum(weighted * mask, dim=1)
        counts = torch.clamp(mask.sum(dim=1), min=1e-9)
        batch_embeddings = (summed / counts).cpu().numpy()

        all_embeddings.append(batch_embeddings)
        pbar.update(1)

    pbar.close()
    return np.vstack(all_embeddings)

In [ ]:
model_name = "Musixmatch/umberto-commoncrawl-cased-v1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForMaskedLM.from_pretrained(model_name, output_hidden_states=True)
model.eval()

In [ ]:
embeddings = generate_weighted_top4_embeddings(
    texts=it_featured_df['cleaned_text'].to_list(),
    model=model,
    tokenizer=tokenizer,
    layer_weights=[0.1, 0.2, 0.3, 0.4],
    max_length=512,
    device="cpu"
)

print(f"Размерность эмбеддингов: {embeddings.shape}")
print(f"Пример эмбеддинга: {embeddings[0][:10]}")

In [ ]:
import pickle

with open("/content/drive/MyDrive/Диплом бакалавриат'25/italian_embeddings.pkl", 'wb') as f:
    pickle.dump(embeddings, f)

In [22]:
with open("/content/drive/MyDrive/Диплом бакалавриат'25/italian_embeddings.pkl", 'rb') as f:
    embeddings = pickle.load(f)

In [23]:
embeddings_np = np.array(embeddings)

In [ ]:
# Проверка нормализации
norms = np.linalg.norm(embeddings_np, axis=1)
print("Минимальная норма:", np.min(norms))
print("Максимальная норма:", np.max(norms))

In [ ]:
# Проверка близости к 1
tolerance = 1e-5
is_normalized = np.allclose(norms, 1.0, atol=tolerance)
print(f"Все эмбеддинги нормализованы (с погрешностью {tolerance})? {is_normalized}")

In [26]:
embeddings_l2 = normalize(embeddings, norm='l2')

#### Снижение размерности

In [ ]:
param_grid = {
    'n_neighbors': np.arange(10, 50, 10),
    'min_dist': [0.01, 0.05],
    'n_components': np.arange(2, 30, 5),
    'metric': ['cosine', 'correlation']
}

In [ ]:
def bootstrap_stability(embeddings, params, n_samples=5, sample_size=0.8):
    stability = []
    for _ in range(n_samples):
        idx = np.random.choice(len(embeddings), int(len(embeddings)*sample_size))
        proj1 = umap.UMAP(**params).fit_transform(embeddings[idx])
        proj2 = umap.UMAP(**params).fit_transform(embeddings[idx])
        stability.append(spearmanr(proj1.flatten(), proj2.flatten())[0])
    return np.mean(stability)

In [ ]:
def optimize_umap_params(embeddings, param_grid, sample_size=1000, random_state=42):
    """
    Оптимизирует параметры UMAP для заданных эмбеддингов с использованием грид-поиска.

    Параметры:
    embeddings (np.array/torch.Tensor): Массив эмбеддингов
    param_grid (dict): Сетка параметров для поиска
    sample_size (int): Размер выборки для ускорения вычислений
    random_state (int): Seed для воспроизводимости

    Возвращает:
    pd.DataFrame: Результаты экспериментов с метриками
    """
    # Сэмплирование данных
    embeddings = embeddings[:sample_size]

    # Стандартизация
    scaler = StandardScaler()
    embeddings_scaled = scaler.fit_transform(embeddings)

    # Основной цикл оптимизации
    results = []
    for params in tqdm(ParameterGrid(param_grid), desc="Grid Search"):
        reducer = umap.UMAP(**params, random_state=random_state)
        projection = reducer.fit_transform(embeddings_scaled)

        # Вычисление метрик
        d_orig = pdist(embeddings_scaled, metric=params['metric'])
        d_proj = pdist(projection, metric='euclidean')

        metrics = {
            'trustworthiness': trustworthiness(embeddings_scaled, projection,
                                             n_neighbors=params['n_neighbors']),
            'stability': bootstrap_stability(embeddings_scaled, params),
            'global_spearman': spearmanr(d_orig, d_proj).correlation
        }

        results.append({**params, **metrics})

    return pd.DataFrame(results)


In [ ]:
results_df = optimize_umap_params(
    embeddings=embeddings_l2,
    param_grid=param_grid,
    sample_size=460
    )

Grid Search: 100%|██████████| 96/96 [25:22<00:00, 15.86s/it]


In [ ]:
def find_best_parameters(results_df, metric_weights=None):
    """
    Находит оптимальные параметры на основе взвешенной комбинации метрик.

    Параметры:
    results_df (pd.DataFrame): Результаты экспериментов
    metric_weights (dict): Веса для метрик (по умолчанию равные веса)

    Возвращает:
    pd.DataFrame: Топ-5 лучших комбинаций с нормализованными оценками
    """
    df = results_df.copy()

    # Задаем веса по умолчанию (равные веса)
    if not metric_weights:
        metric_weights = {
            'trustworthiness': 0.5,
            'stability': 0.3,
            'global_spearman': 0.2
        }

    # Нормализуем метрики (Min-Max scaling)
    for metric in metric_weights:
        df[metric + '_norm'] = (df[metric] - df[metric].min()) / (df[metric].max() - df[metric].min())

    # Рассчитываем общий score
    df['total_score'] = sum(
        df[metric + '_norm'] * weight
        for metric, weight in metric_weights.items()
    )

    # Сортируем по убыванию общего score
    best_results = df.sort_values('total_score', ascending=False).head(5)

    return best_results

best_df = find_best_parameters(results_df)

best_df[['n_neighbors', 'min_dist', 'n_components', 'metric',
               'trustworthiness', 'stability', 'global_spearman', 'total_score']]

,n_neighbors,min_dist,n_components,metric,trustworthiness,stability,global_spearman,total_score
44,10,0.05,27,cosine,0.869044,0.841935,0.485025,0.817485
84,10,0.05,17,correlation,0.871882,0.811681,0.481386,0.816966
64,10,0.01,22,correlation,0.870132,0.862210,0.470050,0.809362
88,10,0.05,22,correlation,0.872342,0.790485,0.482010,0.809104
36,10,0.05,17,cosine,0.870148,0.844489,0.473403,0.805252


In [ ]:
param_grid = {
    'n_neighbors': [10],
    'min_dist': [0.05],
    'n_components': [27],
    'metric': ['cosine']
}

In [ ]:
results_df_1 = optimize_umap_params(
    embeddings=embeddings_l2,
    param_grid=param_grid,
    sample_size=len(embeddings_np)
    )

Grid Search: 100%|██████████| 1/1 [04:03<00:00, 243.13s/it]


In [ ]:
param_grid = {
    'n_neighbors': [10],
    'min_dist': [0.05],
    'n_components': [17],
    'metric': ['correlation']
}

In [ ]:
results_df_2 = optimize_umap_params(
    embeddings=embeddings_l2,
    param_grid=param_grid,
    sample_size=len(embeddings_np)
    )

Grid Search: 100%|██████████| 1/1 [04:17<00:00, 257.43s/it]


In [ ]:
param_grid = {
    'n_neighbors': [10],
    'min_dist': [0.01],
    'n_components': [22],
    'metric': ['correlation']
}

In [ ]:
results_df_3 = optimize_umap_params(
    embeddings=embeddings_l2,
    param_grid=param_grid,
    sample_size=len(embeddings_np)
    )

Grid Search: 100%|██████████| 1/1 [04:28<00:00, 268.72s/it]


In [ ]:
results_best = pd.concat([results_df_1, results_df_2, results_df_3], axis=0)
results_best

,metric,min_dist,n_components,n_neighbors,trustworthiness,stability,global_spearman
0,cosine,0.05,27,10,0.908971,0.991524,0.440600
0,correlation,0.05,17,10,0.908189,0.899680,0.443168
0,correlation,0.01,22,10,0.906756,0.883493,0.431101


In [27]:
scaler = StandardScaler()
scaled_embeds = scaler.fit_transform(embeddings_l2)

In [28]:
reducer = umap.UMAP(
    n_components=27,
    n_neighbors=10,
    min_dist=0.05,
    metric='cosine',
    random_state=42
)

In [29]:
umap_embeds = reducer.fit_transform(scaled_embeds)

#### Кластеризация контекстов

In [31]:
class MetaphorClusteringEnsemble:
    """
    Консенсусная кластеризация для выявления метафорических употреблений глаголов
    """

    def __init__(self, data, algorithm_weights=None):
        """
        Инициализация класса

        Parameters:
        -----------
        data : array-like
            Матрица признаков (векторные представления употреблений)
        algorithm_weights : dict
            Веса для каждого алгоритма в консенсусной матрице
        """
        self.data = np.array(data)
        self.n_samples = self.data.shape[0]

        # Веса алгоритмов (HDBSCAN как самый надежный)
        self.weights = algorithm_weights or {
            'hdbscan': 0.5,
            'dbscan': 0.3,
            'spectral': 0.2
        }

        self.best_params = {}
        self.clusterings = {}
        self.consensus_matrix = None
        self.final_clusters = None

    def optimize_dbscan(self, eps_range=None, min_samples_range=None):
        """
        Подбор оптимальных параметров для DBSCAN
        """
        print("Оптимизация параметров DBSCAN...")

        if eps_range is None:
            # Автоматическое определение диапазона eps на основе k-distance
            k = 4  # минимальное количество точек для формирования кластера
            nbrs = NearestNeighbors(n_neighbors=k).fit(self.data)
            distances, indices = nbrs.kneighbors(self.data)
            distances = np.sort(distances[:, k-1], axis=0)

            # Используем метод "локтя" для определения оптимального eps
            knee_point = self._find_knee_point(distances)
            eps_range = np.linspace(distances[knee_point] * 0.5, distances[knee_point] * 2, 20)

        if min_samples_range is None:
            min_samples_range = range(3, min(20, self.n_samples // 10))

        best_score = -1
        best_params = {}

        param_grid = ParameterGrid({
            'eps': eps_range,
            'min_samples': min_samples_range
        })

        for params in param_grid:
            try:
                dbscan = DBSCAN(**params)
                labels = dbscan.fit_predict(self.data)

                # Проверяем, что есть хотя бы 2 кластера (исключая шум)
                n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
                if n_clusters < 2:
                    continue

                # Оценка качества (без учета точек шума для silhouette)
                mask = labels != -1
                if np.sum(mask) < 2:
                    continue

                score = silhouette_score(self.data[mask], labels[mask])

                if score > best_score:
                    best_score = score
                    best_params = params.copy()

            except:
                continue

        self.best_params['dbscan'] = best_params
        print(f"Лучшие параметры DBSCAN: {best_params}, Score: {best_score:.3f}")

    def optimize_hdbscan(self, min_cluster_size_range=None, min_samples_range=None):
        """
        Подбор оптимальных параметров для HDBSCAN
        """
        print("Оптимизация параметров HDBSCAN...")

        if min_cluster_size_range is None:
            min_cluster_size_range = np.arange(3, 50, 7)

        if min_samples_range is None:
            min_samples_range = np.arange(1, 20, 5)

        best_score = -1
        best_params = {}

        param_grid = ParameterGrid({
            'min_cluster_size': min_cluster_size_range,
            'min_samples': min_samples_range,
            'cluster_selection_epsilon': [0.0, 0.1, 0.2, 0.5]
        })

        for params in param_grid:
            try:
                clusterer = hdbscan.HDBSCAN(**params)
                labels = clusterer.fit_predict(self.data)

                n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
                if n_clusters < 2:
                    continue

                mask = labels != -1
                if np.sum(mask) < 2:
                    continue

                # Комбинированная метрика: silhouette + DBCV (если доступно)
                silhouette = silhouette_score(self.data[mask], labels[mask])

                # HDBSCAN предоставляет собственную метрику качества
                try:
                    dbcv_score = clusterer.relative_validity_
                    if dbcv_score is not None:
                        score = 0.4 * silhouette + 0.6 * dbcv_score
                    else:
                        score = silhouette
                except:
                    score = silhouette

                if score > best_score:
                    best_score = score
                    best_params = params.copy()

            except:
                continue

        self.best_params['hdbscan'] = best_params
        print(f"Лучшие параметры HDBSCAN: {best_params}, Score: {best_score:.3f}")

    def optimize_spectral(self, max_clusters=None):
        """
        Подбор оптимального количества кластеров для Spectral Clustering
        """
        print("Оптимизация параметров Spectral Clustering...")

        if max_clusters is None:
            max_clusters = min(20, self.n_samples // 3)

        # Анализ собственных значений лапласиана для определения количества кластеров
        similarity_matrix = self._compute_similarity_matrix()
        L = laplacian(similarity_matrix, normed=True)
        eigenvals = np.linalg.eigvals(L.toarray() if hasattr(L, 'toarray') else L)
        eigenvals = np.sort(eigenvals)

        # Поиск наибольшего разрыва в собственных значениях
        gaps = np.diff(eigenvals[:max_clusters])
        spectral_suggestion = np.argmax(gaps) + 2

        best_score = -1
        best_params = {}

        # Тестируем диапазон кластеров вокруг предложенного спектральным анализом
        k_range = range(max(2, spectral_suggestion - 3),
                       min(max_clusters + 1, spectral_suggestion + 5))

        for n_clusters in k_range:
            try:
                for affinity in ['rbf', 'nearest_neighbors']:
                    spectral = SpectralClustering(
                        n_clusters=n_clusters,
                        affinity=affinity,
                        random_state=42
                    )
                    labels = spectral.fit_predict(self.data)

                    score = silhouette_score(self.data, labels)

                    if score > best_score:
                        best_score = score
                        best_params = {
                            'n_clusters': n_clusters,
                            'affinity': affinity
                        }

            except:
                continue

        self.best_params['spectral'] = best_params
        print(f"Лучшие параметры Spectral: {best_params}, Score: {best_score:.3f}")

    def _compute_similarity_matrix(self):
        """Вычисление матрицы сходства для спектральной кластеризации"""
        distances = pdist(self.data, metric='euclidean')
        distance_matrix = squareform(distances)

        # RBF kernel
        gamma = 1.0 / (2 * np.var(distances))
        similarity_matrix = np.exp(-gamma * distance_matrix ** 2)

        return similarity_matrix

    def _find_knee_point(self, values):
        """Поиск точки 'колена' в отсортированном массиве"""
        n_points = len(values)
        all_coords = np.vstack((range(n_points), values)).T

        # Первая и последняя точки
        first_point = all_coords[0]
        last_point = all_coords[-1]

        # Вектор от первой к последней точке
        line_vec = last_point - first_point
        line_vec_norm = line_vec / np.sqrt(np.sum(line_vec**2))

        # Векторы от первой точки к каждой точке
        vec_from_first = all_coords - first_point

        # Проекции на линию
        scalar_proj = np.dot(vec_from_first, line_vec_norm.reshape(-1, 1)).flatten()
        vec_proj = np.outer(scalar_proj, line_vec_norm)

        # Перпендикулярные расстояния
        distances = np.sqrt(np.sum((vec_from_first - vec_proj)**2, axis=1))

        return np.argmax(distances)

    def fit_all_algorithms(self):
        """Обучение всех алгоритмов с оптимальными параметрами"""
        print("\nОбучение алгоритмов кластеризации...")

        # DBSCAN
        if 'dbscan' in self.best_params:
            dbscan = DBSCAN(**self.best_params['dbscan'])
            self.clusterings['dbscan'] = dbscan.fit_predict(self.data)
            print(f"DBSCAN: {len(set(self.clusterings['dbscan'])) - (1 if -1 in self.clusterings['dbscan'] else 0)} кластеров")

        # HDBSCAN
        if 'hdbscan' in self.best_params:
            hdbscan_clusterer = hdbscan.HDBSCAN(**self.best_params['hdbscan'])
            self.clusterings['hdbscan'] = hdbscan_clusterer.fit_predict(self.data)
            print(f"HDBSCAN: {len(set(self.clusterings['hdbscan'])) - (1 if -1 in self.clusterings['hdbscan'] else 0)} кластеров")

        # Spectral Clustering
        if 'spectral' in self.best_params:
            spectral = SpectralClustering(**self.best_params['spectral'], random_state=42)
            self.clusterings['spectral'] = spectral.fit_predict(self.data)
            print(f"Spectral: {self.best_params['spectral']['n_clusters']} кластеров")

    def compute_consensus_matrix(self):
        """Вычисление взвешенной консенсусной матрицы"""
        print("\nВычисление консенсусной матрицы...")

        self.consensus_matrix = np.zeros((self.n_samples, self.n_samples))
        total_weight = 0

        for alg_name, labels in self.clusterings.items():
            if alg_name not in self.weights:
                continue

            weight = self.weights[alg_name]
            total_weight += weight

            # Создание матрицы совместного вхождения для текущего алгоритма
            cooccurrence_matrix = np.zeros((self.n_samples, self.n_samples))

            for i in range(self.n_samples):
                for j in range(i, self.n_samples):
                    # Проверяем, что обе точки не являются шумом (-1)
                    if labels[i] != -1 and labels[j] != -1 and labels[i] == labels[j]:
                        cooccurrence_matrix[i, j] = 1
                        cooccurrence_matrix[j, i] = 1

            self.consensus_matrix += weight * cooccurrence_matrix

        # Нормализация на общий вес
        self.consensus_matrix /= total_weight

        print("Консенсусная матрица вычислена")

    def extract_final_clusters(self, threshold=0.5):
        """
        Извлечение финальных консенсусных кластеров

        Parameters:
        -----------
        threshold : float
            Порог для определения принадлежности к одному кластеру
        """
        print(f"\nИзвлечение финальных кластеров (порог: {threshold})...")

        # Создание графа связей на основе консенсусной матрицы
        adjacency_matrix = (self.consensus_matrix >= threshold).astype(int)

        # Поиск связанных компонент (кластеров)
        visited = np.zeros(self.n_samples, dtype=bool)
        clusters = []
        cluster_id = 0
        self.final_clusters = np.full(self.n_samples, -1)  # -1 для неназначенных точек

        def dfs(node, current_cluster):
            """Поиск в глубину для нахождения связанных компонент"""
            visited[node] = True
            current_cluster.append(node)

            for neighbor in range(self.n_samples):
                if adjacency_matrix[node, neighbor] and not visited[neighbor]:
                    dfs(neighbor, current_cluster)

        for i in range(self.n_samples):
            if not visited[i]:
                current_cluster = []
                dfs(i, current_cluster)

                # Сохраняем только кластеры с более чем одной точкой
                if len(current_cluster) > 1:
                    clusters.append(current_cluster)
                    for point in current_cluster:
                        self.final_clusters[point] = cluster_id
                    cluster_id += 1

        n_clusters = len(clusters)
        n_noise = np.sum(self.final_clusters == -1)

        print(f"Найдено {n_clusters} консенсусных кластеров")
        print(f"Точек шума: {n_noise}")

        # Статистика по размерам кластеров
        if n_clusters > 0:
            cluster_sizes = [len(cluster) for cluster in clusters]
            print(f"Размеры кластеров: мин={min(cluster_sizes)}, макс={max(cluster_sizes)}, средний={np.mean(cluster_sizes):.1f}")

        return self.final_clusters

    def fit(self, dbscan_params=None, hdbscan_params=None, spectral_params=None,
            consensus_threshold=0.5):
        """
        Полный пайплайн кластеризации

        Parameters:
        -----------
        dbscan_params, hdbscan_params, spectral_params : dict
            Параметры для оптимизации каждого алгоритма
        consensus_threshold : float
            Порог для извлечения финальных кластеров
        """
        print("=== ЗАПУСК КОНСЕНСУСНОЙ КЛАСТЕРИЗАЦИИ ===")

        # Оптимизация параметров
        self.optimize_dbscan(**(dbscan_params or {}))
        self.optimize_hdbscan(**(hdbscan_params or {}))
        self.optimize_spectral(**(spectral_params or {}))

        # Обучение алгоритмов
        self.fit_all_algorithms()

        # Построение консенсуса
        self.compute_consensus_matrix()

        # Извлечение финальных кластеров
        final_clusters = self.extract_final_clusters(threshold=consensus_threshold)

        return final_clusters

In [32]:
def visualize_consensus_clusters(data, clusters, method='both', figsize=(16, 7),
                               save_path=None, dpi=300):
    """
    Визуализация результатов консенсусной кластеризации

    Parameters:
    -----------
    data : np.ndarray
        Исходные данные (векторные представления)
    clusters : np.ndarray
        Метки кластеров (-1 для шума)
    method : str
        Метод снижения размерности: 'pca', 'tsne', или 'both'
    figsize : tuple
        Размер фигуры
    save_path : str
        Путь для сохранения графика
    dpi : int
        Разрешение для сохранения
    """

    # Настройка стиля и цветов
    sns.set(style="whitegrid", font_scale=1.0)
    palette = sns.color_palette("husl", 8)
    ITALIAN_COLOR = palette[0]

    # Получаем уникальные кластеры (исключаем шум -1)
    unique_clusters = np.unique(clusters)
    noise_mask = clusters == -1
    n_clusters = len(unique_clusters) - (1 if -1 in unique_clusters else 0)

    # Создаем цветовую палитру
    cluster_colors = sns.color_palette("husl", n_colors=max(n_clusters, 8))
    if n_clusters > 0:
        cluster_colors[0] = ITALIAN_COLOR  # Первый кластер всегда итальянский

    # Настройка параметров визуализации
    POINT_ALPHA = 0.85
    NOISE_ALPHA = 0.5
    BORDER_ALPHA = 0.9
    ANNOTATION_FONT = {'size': 12, 'weight': 'semibold', 'color': 'white'}

    # Создаем фигуру
    if method == 'both':
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=figsize, facecolor='white')
        axes = [ax1, ax2]
        methods = ['PCA', 't-SNE']
    else:
        fig, ax = plt.subplots(figsize=(figsize[0]//2, figsize[1]), facecolor='white')
        axes = [ax]
        methods = ['PCA' if method == 'pca' else 't-SNE']

    # Применяем методы снижения размерности
    reducers = []
    if 'PCA' in methods:
        pca = PCA(n_components=2, random_state=42)
        data_pca = pca.fit_transform(data)
        reducers.append(('PCA', data_pca, pca))

    if 't-SNE' in methods:
        tsne = TSNE(n_components=2, random_state=42,
                   perplexity=min(30, len(data)//4), init='pca')
        data_tsne = tsne.fit_transform(data)
        reducers.append(('t-SNE', data_tsne, None))

    # Создаем визуализации
    for idx, (reduction_name, data_2d, reducer) in enumerate(reducers):
        ax = axes[idx]
        ax.set_facecolor('white')

        # Рисуем шум (если есть)
        if np.any(noise_mask):
            ax.scatter(data_2d[noise_mask, 0], data_2d[noise_mask, 1],
                      c='#d9d9d9', alpha=NOISE_ALPHA, s=25,
                      marker='.', edgecolors='none',
                      label='Шум')

        # Визуализация кластеров
        cluster_counter = 1  # Начинаем нумерацию с 1
        for cluster_id in unique_clusters:
            if cluster_id == -1: continue

            mask = clusters == cluster_id
            cluster_points = data_2d[mask]
            color = cluster_colors[(cluster_counter-1) % len(cluster_colors)]

            # Основные точки кластера
            ax.scatter(cluster_points[:, 0], cluster_points[:, 1],
                     c=[color], alpha=POINT_ALPHA, s=65,
                     edgecolors='w', linewidth=0.6,
                     label=f'Кластер {cluster_counter}', zorder=3)

            # Границы кластера
            if len(cluster_points) >= 3:
                try:
                    hull = ConvexHull(cluster_points)
                    hull_points = cluster_points[hull.vertices]

                    # Сглаженные границы
                    poly = mpatches.Polygon(hull_points, closed=True,
                                          fill=False, edgecolor=color,
                                          linewidth=1.8, linestyle='-',
                                          alpha=BORDER_ALPHA, zorder=2)
                    ax.add_patch(poly)
                except:
                    if len(cluster_points) >= 2:
                        cov = np.cov(cluster_points.T)
                        vals, vecs = np.linalg.eigh(cov)
                        angle = np.degrees(np.arctan2(*vecs[:, 0][::-1]))
                        w, h = 3 * np.sqrt(vals)

                        ellipse = Ellipse(cluster_points.mean(0), w, h, angle,
                                         fill=False, edgecolor=color,
                                         linewidth=1.8, alpha=BORDER_ALPHA)
                        ax.add_patch(ellipse)

            # Аннотация кластера
            center = cluster_points.mean(axis=0)
            ax.text(center[0], center[1], str(cluster_counter),
                   ha='center', va='center', **ANNOTATION_FONT,
                   bbox=dict(boxstyle="circle,pad=0.3",
                            facecolor=color, alpha=0.9,
                            edgecolor='none'))

            cluster_counter += 1

        # Заголовки и подзаголовки
        ax.set_title(f'Консенсусные кластеры\n(Метод проекции: {reduction_name})',
                    fontsize=13, pad=12)

        # Оформление осей
        ax.set_xlabel(f'{reduction_name} 1', labelpad=10)
        ax.set_ylabel(f'{reduction_name} 2', labelpad=10)
        ax.tick_params(axis='both', which='both', length=0)

        # Добавление легенды
        if idx == 0:
            handles = [plt.Line2D([0], [0], marker='o', color='w',
                      markerfacecolor=c, markersize=10, alpha=0.8)
                     for i, c in enumerate(cluster_colors) if i < n_clusters]
            labels = [f'Кластер {i+1}' for i in range(n_clusters)]
            if np.any(noise_mask):
                handles.append(plt.Line2D([0], [0], marker='.', color='w',
                              markerfacecolor='#d9d9d9', markersize=10))
                labels.append('Шум')

            legend = ax.legend(handles, labels, loc='upper right',
                             bbox_to_anchor=(1.15, 1.02),
                             frameon=True, framealpha=0.95,
                             borderpad=1)
            legend.get_frame().set_edgecolor('#333333')

        # Объясненная дисперсия для PCA
        if reduction_name == 'PCA' and reducer is not None:
            explained_var = reducer.explained_variance_ratio_
            ax.text(0.03, 0.97,
                   f'Expl. Variance:\nPC1: {explained_var[0]:.1%}\nPC2: {explained_var[1]:.1%}',
                   transform=ax.transAxes, ha='left', va='top',
                   fontsize=10, bbox=dict(facecolor='white', alpha=0.8,
                                       edgecolor='none', pad=6))

    # Общий заголовок
    if method == 'both':
        plt.suptitle('Результаты консенсусной кластеризации',
                   fontsize=16, y=1.02, weight='bold')

    # Финализация
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])

    if save_path:
        plt.savefig(save_path, dpi=dpi, bbox_inches='tight',
                   facecolor='white', transparent=False)
        print(f"График сохранён: {save_path}")

    plt.show()

    # Вывод статистики
    print("\n=== СТАТИСТИКА ===")
    print(f"Кластеров: {n_clusters}")
    print(f"Шум: {noise_mask.sum()} точек")
    print(f"Всего данных: {len(data)}")

    if n_clusters > 0:
        sizes = [np.sum(clusters == c) for c in unique_clusters if c != -1]
        print(f"\nРазмеры кластеров:")
        print(pd.Series(sizes).describe().to_string())

In [33]:
def analyze_cluster_medoids(data, clusters, contexts=None, top_k=20,
                           distance_metric='euclidean', return_indices=False):
    """
    Анализ медоидов кластеров и поиск ближайших к ним контекстов

    Parameters:
    -----------
    data : np.ndarray
        Матрица признаков (векторные представления)
    clusters : np.ndarray
        Метки кластеров для каждой точки (-1 для шума)
    contexts : list or pd.Series, optional
        Список контекстов/предложений соответствующих каждой точке данных
    top_k : int, default=20
        Количество ближайших контекстов для вывода
    distance_metric : str, default='euclidean'
        Метрика расстояния для поиска медоида
    return_indices : bool, default=False
        Возвращать ли индексы медоидов и ближайших точек

    Returns:
    --------
    dict : Словарь с результатами анализа для каждого кластера
    """

    # Проверяем входные данные
    if len(data) != len(clusters):
        raise ValueError("Размеры data и clusters должны совпадать")

    if contexts is not None and len(contexts) != len(data):
        raise ValueError("Размеры contexts и data должны совпадать")

    # Получаем уникальные кластеры (исключаем шум -1)
    unique_clusters = np.unique(clusters)
    valid_clusters = unique_clusters[unique_clusters != -1]

    results = {}

    print("=== АНАЛИЗ МЕДОИДОВ КЛАСТЕРОВ ===\n")

    for cluster_id in valid_clusters:
        print(f" КЛАСТЕР {cluster_id}")
        print("=" * 50)

        # Получаем точки текущего кластера
        cluster_mask = clusters == cluster_id
        cluster_data = data[cluster_mask]
        cluster_indices = np.where(cluster_mask)[0]
        cluster_size = len(cluster_data)

        print(f"Размер кластера: {cluster_size} точек")

        # Находим медоид - точку с минимальной суммой расстояний до всех остальных в кластере
        if cluster_size == 1:
            medoid_idx_in_cluster = 0
            medoid_distance_sum = 0.0
        else:
            # Вычисляем матрицу расстояний внутри кластера
            intra_distances = pairwise_distances(cluster_data, metric=distance_metric)

            # Находим точку с минимальной суммой расстояний (медоид)
            distance_sums = np.sum(intra_distances, axis=1)
            medoid_idx_in_cluster = np.argmin(distance_sums)
            medoid_distance_sum = distance_sums[medoid_idx_in_cluster]

        # Индекс медоида в исходных данных
        medoid_global_idx = cluster_indices[medoid_idx_in_cluster]
        medoid_vector = data[medoid_global_idx]

        print(f"Медоид: точка #{medoid_global_idx}")
        print(f"Средняя внутрикластерная дистанция медоида: {medoid_distance_sum / cluster_size:.4f}")

        # Показываем контекст медоида, если есть
        if contexts is not None:
            print(f"\n КОНТЕКСТ МЕДОИДА:")
            print(f"   \"{contexts[medoid_global_idx]}\"\n")

        # Находим top_k ближайших точек к медоиду из всего датасета
        distances_to_medoid = cdist([medoid_vector], data, metric=distance_metric)[0]

        # Сортируем по расстоянию и берем top_k
        nearest_indices = np.argsort(distances_to_medoid)[:top_k]
        nearest_distances = distances_to_medoid[nearest_indices]

        print(f"ТОП-{top_k} БЛИЖАЙШИХ КОНТЕКСТОВ К МЕДОИДУ:")
        print("-" * 60)

        # Сохраняем результаты для кластера
        cluster_results = {
            'medoid_index': medoid_global_idx,
            'medoid_vector': medoid_vector,
            'cluster_size': cluster_size,
            'avg_intra_distance': medoid_distance_sum / cluster_size,
            'nearest_indices': nearest_indices,
            'nearest_distances': nearest_distances
        }

        if contexts is not None:
            cluster_results['medoid_context'] = contexts[medoid_global_idx]
            cluster_results['nearest_contexts'] = [contexts[idx] for idx in nearest_indices]

            # Выводим ближайшие контексты
            for rank, (idx, dist) in enumerate(zip(nearest_indices, nearest_distances), 1):
                # Отмечаем, к какому кластеру принадлежит каждая точка
                point_cluster = clusters[idx]
                cluster_marker = f"[К{point_cluster}]" if point_cluster != -1 else "[ШУМ]"

                # Особо выделяем медоид
                is_medoid = "МЕДОИД" if idx == medoid_global_idx else ""

                print(f"{rank:2d}. {cluster_marker} (дист: {dist:.4f}) {is_medoid}")
                print(f"    \"{contexts[idx]}\"")
                print()
        else:
            # Если контекстов нет, просто показываем индексы и расстояния
            for rank, (idx, dist) in enumerate(zip(nearest_indices, nearest_distances), 1):
                point_cluster = clusters[idx]
                cluster_marker = f"[К{point_cluster}]" if point_cluster != -1 else "[ШУМ]"
                is_medoid = "МЕДОИД" if idx == medoid_global_idx else ""

                print(f"{rank:2d}. Точка #{idx} {cluster_marker} (дист: {dist:.4f}) {is_medoid}")

        results[cluster_id] = cluster_results
        print("\n" + "="*80 + "\n")

    # Выводим общую статистику
    print("ОБЩАЯ СТАТИСТИКА ПО КЛАСТЕРАМ")
    print("=" * 50)

    for cluster_id, result in results.items():
        print(f"Кластер {cluster_id}: {result['cluster_size']} точек, "
              f"средняя дистанция до медоида: {result['avg_intra_distance']:.4f}")

    if return_indices:
        return results
    else:
        # Возвращаем упрощенную версию без векторов для удобства
        simple_results = {}
        for cluster_id, result in results.items():
            simple_results[cluster_id] = {
                'medoid_index': result['medoid_index'],
                'cluster_size': result['cluster_size'],
                'avg_intra_distance': result['avg_intra_distance'],
                'nearest_indices': result['nearest_indices'].tolist(),
                'nearest_distances': result['nearest_distances'].tolist()
            }
            if contexts is not None:
                simple_results[cluster_id]['medoid_context'] = result['medoid_context']
                simple_results[cluster_id]['nearest_contexts'] = result['nearest_contexts']

        return simple_results

def compare_medoids_similarity(results, data, distance_metric='euclidean'):
    """
    Дополнительная функция для анализа схожести между медоидами разных кластеров

    Parameters:
    -----------
    results : dict
        Результаты функции analyze_cluster_medoids с return_indices=True
    data : np.ndarray
        Исходная матрица данных
    distance_metric : str
        Метрика для вычисления расстояний
    """

    if len(results) < 2:
        print("Недостаточно кластеров для сравнения медоидов")
        return

    print("\n МАТРИЦА РАССТОЯНИЙ МЕЖДУ МЕДОИДАМИ")
    print("=" * 60)

    cluster_ids = list(results.keys())
    medoid_vectors = np.array([results[cid]['medoid_vector'] for cid in cluster_ids])

    # Вычисляем матрицу расстояний между медоидами
    medoid_distances = pairwise_distances(medoid_vectors, metric=distance_metric)

    # Создаем красивую таблицу
    df_distances = pd.DataFrame(medoid_distances,
                               index=[f"К{cid}" for cid in cluster_ids],
                               columns=[f"К{cid}" for cid in cluster_ids])

    print(df_distances.round(4))

    # Находим самые близкие и далекие пары
    mask = np.triu(np.ones_like(medoid_distances, dtype=bool), k=1)
    upper_distances = medoid_distances[mask]

    if len(upper_distances) > 0:
        min_dist_idx = np.unravel_index(np.argmin(medoid_distances + np.eye(len(cluster_ids)) * 1000),
                                       medoid_distances.shape)
        max_dist_idx = np.unravel_index(np.argmax(medoid_distances), medoid_distances.shape)

        print(f"Самые близкие кластеры: К{cluster_ids[min_dist_idx[0]]} ↔ К{cluster_ids[min_dist_idx[1]]} "
              f"(расстояние: {medoid_distances[min_dist_idx]:.4f})")
        print(f"Самые далекие кластеры: К{cluster_ids[max_dist_idx[0]]} ↔ К{cluster_ids[max_dist_idx[1]]} "
              f"(расстояние: {medoid_distances[max_dist_idx]:.4f})")

In [51]:
embeddings_list = umap_embeds.tolist()

if len(embeddings_list) == len(it_featured_df):
    it_featured_df['embeddings'] = embeddings_list
else:
    print("Количество эмбеддингов не совпадает с количеством строк в DataFrame.")

In [ ]:
ensemble = MetaphorClusteringEnsemble(
    data=np.array(it_featured_df[it_featured_df['found_verbs'] == 'spingere (толкнуть)']['embeddings'].tolist()),
    algorithm_weights={
          'hdbscan': 0.5,  # Наибольший вес для HDBSCAN как самого надежного
          'dbscan': 0.2,
          'spectral': 0.3
      }
)

final_clusters = ensemble.fit(consensus_threshold=0.4)

In [ ]:
visualize_consensus_clusters(
     data=np.array(it_featured_df[it_featured_df['found_verbs'] == 'spingere (толкнуть)']['embeddings'].tolist()),
     clusters=final_clusters,
     method='both'
)

In [ ]:
detailed_results = analyze_cluster_medoids(
    data=np.array(it_featured_df[it_featured_df['found_verbs'] == 'spingere (толкнуть)']['embeddings'].tolist()),
    clusters=final_clusters,
    contexts=it_featured_df[it_featured_df['found_verbs'] == 'spingere (толкнуть)']['context'].tolist(),
    return_indices=True
)
compare_medoids_similarity(detailed_results, np.array(it_featured_df[it_featured_df['found_verbs'] == 'spingere (толкнуть)']['embeddings'].tolist()))

## English Tweets

In [ ]:
english_df = pd.read_csv("/content/drive/MyDrive/Диплом бакалавриат'25/final_english_7to23.csv")

In [ ]:
english_df['found_verbs'].value_counts()

,count
found_verbs,
hit (ударить),9999
push (толкнуть),5633
beat (ударить),4456
kick (пнуть),4261
strike (ударить),2067
touch (тронуть),2025
crash (разбиться),823
tap (постучать),808
punch (ударить),510


In [ ]:
def sample_verbs(df: pd.DataFrame, threshold: int = 460) -> pd.DataFrame:
    """
    Извлекает случайные контексты для глаголов, которые встречаются более заданного количества раз.

    :param df: DataFrame с данными
    :param threshold: Минимальное количество вхождений для выбора глагола
    :return: DataFrame с отобранными контекстами
    """

    verb_counts = df['found_verbs'].value_counts()

    common_verbs = verb_counts[verb_counts > threshold].index

    sampled_contexts_list = []

    for verb in common_verbs:
        contexts = df[df['found_verbs'] == verb]

        sampled_contexts = contexts.sample(n=min(threshold, len(contexts)), random_state=42)
        sampled_contexts_list.append(sampled_contexts)

    return pd.concat(sampled_contexts_list, ignore_index=True)


In [ ]:
english_contexts = sample_verbs(english_df)

In [ ]:
def enumerate_verbs(df):
    df = df.sort_values(by='found_verbs').reset_index(drop=True)

    df['context_id'] = df.groupby('found_verbs').cumcount() + 1

    df['context_id'] = df['found_verbs'] + '_' + df['context_id'].astype(str)

    return df

english_contexts = enumerate_verbs(english_contexts)

In [ ]:
english_contexts = english_contexts[['id', 'context_id', 'user', 'date', 'tweet', 'context', 'found_verbs', 'spacy_word_count']]
english_contexts['found_verbs'].value_counts()

,count
found_verbs,
beat (ударить),460
crash (разбиться),460
hit (ударить),460
kick (пнуть),460
punch (ударить),460
push (толкнуть),460
slap (пощечина),460
strike (ударить),460
tap (постучать),460


In [ ]:
english_contexts.to_csv("/content/drive/MyDrive/Диплом бакалавриат'25/english_contexts.csv")

### Sentence-RoBERTa

In [ ]:
model_name = 'sentence-transformers/all-roberta-large-v1'
model = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

#### Предобработка текста

In [13]:
english_df = pd.read_csv("/content/drive/MyDrive/Диплом бакалавриат'25/english_contexts.csv")
english_df = english_df[['id', 'context_id', 'user', 'date', 'tweet', 'context', 'found_verbs', 'spacy_word_count']]

In [14]:
def preprocess_tweet(tweet):
    # Удаление URL
    tweet = re.sub(r'http\S+', '', tweet)
    # Замена упоминаний
    tweet = re.sub(r'@\w+', '@user', tweet)
    # Обработка хэштегов
    tweet = re.sub(r'#(\w+)', lambda x: x.group(1), tweet)
    return tweet.strip()

english_df['cleaned_text'] = english_df['context'].apply(preprocess_tweet)

#### Создание эмбеддингов

In [15]:
nlp = spacy.load("en_core_web_sm")
target_verbs = np.unique([verb.split()[0] for verb in english_df['found_verbs'].tolist()])

def extract_verbs(df):
    """Извлечение целевых глаголов из оригинального текста"""
    verb_records = []

    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Extracting verbs"):
        doc = nlp(row['context'])
        verbs = []

        for token in doc:
            if token.pos_ == "VERB":
                for target_verb in target_verbs:
                    # Проверка частичного/полного совпадения
                    if target_verb in token.lemma_ or token.lemma_ in target_verb:
                        verbs.append({
                            'raw_text': token.text,  # Оригинальная словоформа
                            'pattern': target_verb
                        })
                        break

        verb_records.append(verbs)

    df['verbs_info'] = verb_records
    return df

In [ ]:
english_df = extract_verbs(english_df)

In [19]:
def create_masked_text(df):
    """Создает отдельные записи для каждого целевого глагола"""

    new_rows = []

    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Processing rows"):
        text = row['cleaned_text']
        verbs = row['verbs_info']

        # Для каждого глагола создаем отдельную маску
        for i, verb_info in enumerate(verbs):
            new_row = row.copy()
            target_verb = verb_info['raw_text']

            # Точная замена только конкретного глагола
            masked_text = re.sub(
                r'\b{}\b'.format(re.escape(target_verb)),
                '<mask>',
                text,
                count=1  # Заменяем только первое вхождение
            )

            # Проверка успешности замены
            if masked_text == text:
                continue  # Пропускаем неудачные замены

            # Обновляем информацию о глаголах
            new_verbs = [v for j, v in enumerate(verbs) if j != i]
            new_row['verbs_info'] = verbs
            new_row['cleaned_text'] = masked_text
            new_row['masked_verb'] = target_verb  # Добавляем новое поле

            new_rows.append(new_row)

    # Создаем новый DataFrame
    return pd.DataFrame(new_rows).reset_index(drop=True)

In [ ]:
en_featured_df = create_masked_text(english_df)

##### Эксперимент с выбором слоев и методов усреднения

In [ ]:
class RoBERTaEmbeddingExperiment:
    """
    Класс для экспериментов с различными методами извлечения эмбеддингов из RoBERTa-large
    """

    def __init__(self, model_name="roberta-large"):
        """Инициализация модели и токенизатора"""
        print("Загрузка модели RoBERTa-large...")
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForMaskedLM.from_pretrained(model_name, output_hidden_states=True)
        self.model.eval()
        self.n_layers = self.model.config.num_hidden_layers  # 24 слоя

        # Загружаем spaCy для анализа POS-тегов
        self.nlp = spacy.load("en_core_web_sm")

        # Определяем методы извлечения эмбеддингов
        self.extraction_methods = {
            # Стандартные методы
            'last_4_avg': self._extract_last_4_average,
            'last_2_avg': self._extract_last_2_average,
            'weighted_top_4': self._extract_weighted_top_4,

            # Средние слои
            'middle_layers_avg': self._extract_middle_layers,
            'attention_weighted_middle': self._extract_attention_weighted_middle,

            # Допонительные методы
            'exponential_decay': self._extract_exponential_decay,
            'semantic_attention': self._extract_semantic_attention,
            'layer_dropout_ensemble': self._extract_layer_dropout_ensemble,
            'dynamic_layer_selection': self._extract_dynamic_layer_selection,
            'contrastive_pooling': self._extract_contrastive_pooling
        }

        print("Модель загружена!")

    def _get_token_embeddings(self, text, return_attention=False):
        """Базовая функция для получения скрытых состояний всех слоев"""
        inputs = self.tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)

        with torch.no_grad():
            if return_attention:
                outputs = self.model(**inputs, output_attentions=True)
                return outputs.hidden_states, outputs.attentions, inputs
            else:
                outputs = self.model(**inputs)
                return outputs.hidden_states, None, inputs

    # ========== СТАНДАРТНЫЕ МЕТОДЫ ==========

    def _extract_last_4_average(self, text):
        """Усреднение последних 4 слоев"""
        hidden_states, _, inputs = self._get_token_embeddings(text)

        # Берем последние 4 слоя
        last_4 = torch.stack(hidden_states[-4:])  # [4, batch, seq_len, hidden_dim]
        averaged = torch.mean(last_4, dim=0)  # [batch, seq_len, hidden_dim]

        # Усредняем по всем токенам (исключая специальные)
        mask = inputs['attention_mask'].bool()
        pooled = self._mean_pooling(averaged, mask)

        return pooled.squeeze().numpy()

    def _extract_last_2_average(self, text):
        """Усреднение последних 2 слоев"""
        hidden_states, _, inputs = self._get_token_embeddings(text)

        last_2 = torch.stack(hidden_states[-2:])
        averaged = torch.mean(last_2, dim=0)

        mask = inputs['attention_mask'].bool()
        pooled = self._mean_pooling(averaged, mask)

        return pooled.squeeze().numpy()

    def _extract_weighted_top_4(self, text):
        """Взвешенное усреднение топ-4 слоев с убывающими весами"""
        hidden_states, _, inputs = self._get_token_embeddings(text)

        # Веса для последних 4 слоев: [0.4, 0.3, 0.2, 0.1]
        weights = torch.tensor([0.1, 0.2, 0.3, 0.4]).unsqueeze(1).unsqueeze(2)

        last_4 = torch.stack(hidden_states[-4:])
        weighted = torch.sum(last_4 * weights, dim=0)

        mask = inputs['attention_mask'].bool()
        pooled = self._mean_pooling(weighted, mask)

        return pooled.squeeze().numpy()

    # ========== СРЕДНИЕ СЛОИ ==========

    def _extract_middle_layers(self, text):
        """Усреднение средних слоев (слои 10-18 из 24)"""
        hidden_states, _, inputs = self._get_token_embeddings(text)

        # Берем средние слои (10-17 включительно)
        middle_layers = torch.stack(hidden_states[10:18])
        averaged = torch.mean(middle_layers, dim=0)

        mask = inputs['attention_mask'].bool()
        pooled = self._mean_pooling(averaged, mask)
        return pooled.squeeze().numpy()

    def _extract_attention_weighted_middle(self, text):
        """Взвешивание средних слоев на основе attention weights"""
        hidden_states, attentions, inputs = self._get_token_embeddings(text, return_attention=True)

        # Берем attention веса средних слоев (10-17)
        middle_attentions = attentions[10:18]
        middle_states = torch.stack(hidden_states[10:18])

        # Усредняем attention по головам и используем как веса
        attention_weights = torch.stack([att.mean(dim=1) for att in middle_attentions])  # [4, batch, seq_len, seq_len]
        attention_weights = attention_weights.mean(dim=-1)  # [4, batch, seq_len]
        attention_weights = torch.softmax(attention_weights.mean(dim=-1), dim=0)  # [4, batch]

        # Применяем веса к состояниям
        weighted_states = torch.sum(middle_states * attention_weights.unsqueeze(-1).unsqueeze(-1), dim=0)

        mask = inputs['attention_mask'].bool()
        pooled = self._mean_pooling(weighted_states, mask)

        return pooled.squeeze().numpy()

    # ========== ДОПОЛНИТЕЛЬНЫЕ МЕТОДЫ ==========

    def _extract_exponential_decay(self, text):
        """Экспоненциальное затухание весов от последних к средним слоям"""
        hidden_states, _, inputs = self._get_token_embeddings(text)

        # Создаем экспоненциально убывающие веса
        num_layers = len(hidden_states)
        decay_factor = 0.8
        weights = torch.tensor([decay_factor ** (num_layers - i - 1) for i in range(num_layers)])
        weights = weights / weights.sum()  # нормализуем

        # Исключаем первые 4 слоя (слишком низкоуровневые)
        relevant_states = torch.stack(hidden_states[4:])
        relevant_weights = weights[4:] / weights[4:].sum()

        weighted = torch.sum(relevant_states * relevant_weights.unsqueeze(1).unsqueeze(2), dim=0)

        mask = inputs['attention_mask'].bool()
        pooled = self._mean_pooling(weighted, mask)

        return pooled.squeeze().numpy()

    def _extract_semantic_attention(self, text):
        """Адаптивное взвешивание на основе семантической важности токенов"""
        hidden_states, attentions, inputs = self._get_token_embeddings(text, return_attention=True)

        # Берем верхние слои
        top_layers = torch.stack(hidden_states[-4:])
        top_attentions = attentions[-4:]

        # Вычисляем "семантическую важность" как дисперсию attention весов
        semantic_importance = []
        for att in top_attentions:
            # att: [batch, num_heads, seq_len, seq_len]
            att_variance = torch.var(att.mean(dim=1), dim=-1)  # [batch, seq_len]
            semantic_importance.append(att_variance.mean(dim=-1))  # [batch]

        semantic_weights = torch.stack(semantic_importance)  # [4, batch]
        semantic_weights = torch.softmax(semantic_weights, dim=0)

        weighted = torch.sum(top_layers * semantic_weights.unsqueeze(-1).unsqueeze(-1), dim=0)

        mask = inputs['attention_mask'].bool()
        pooled = self._mean_pooling(weighted, mask)

        return pooled.squeeze().numpy()

    def _extract_layer_dropout_ensemble(self, text):
        """Ансамбль с случайным исключением слоев"""
        hidden_states, _, inputs = self._get_token_embeddings(text)

        # Создаем несколько ансамблей с разными комбинациями слоев
        ensembles = []
        relevant_layers = list(range(12, len(hidden_states)))  # средние и верхние слои

        for _ in range(5):  # 5 разных ансамблей
            # Случайно выбираем 60-80% слоев
            num_select = np.random.randint(int(len(relevant_layers) * 0.6), int(len(relevant_layers) * 0.8) + 1)
            selected_indices = np.random.choice(relevant_layers, num_select, replace=False)

            selected_states = torch.stack([hidden_states[i] for i in selected_indices])
            ensemble_avg = torch.mean(selected_states, dim=0)
            ensembles.append(ensemble_avg)

        # Усредняем все ансамбли
        final_ensemble = torch.mean(torch.stack(ensembles), dim=0)

        mask = inputs['attention_mask'].bool()
        pooled = self._mean_pooling(final_ensemble, mask)

        return pooled.squeeze().numpy()

    def _extract_dynamic_layer_selection(self, text):
        """Динамический выбор слоев на основе косинусного сходства"""
        hidden_states, _, inputs = self._get_token_embeddings(text)

        # Вычисляем попарное косинусное сходство между слоями
        layer_similarities = []
        for i in range(6, len(hidden_states) - 1):  # средние и верхние слои
            state1 = hidden_states[i].mean(dim=1)  # [batch, hidden_dim]
            state2 = hidden_states[i + 1].mean(dim=1)

            similarity = torch.cosine_similarity(state1, state2, dim=-1)
            layer_similarities.append(similarity.item())

        # Выбираем слои с наименьшим сходством (наиболее информативные переходы)
        similarity_scores = np.array(layer_similarities)
        diverse_indices = np.argsort(similarity_scores)[:4]  # топ-4 наиболее различающихся
        diverse_indices = [i + 6 for i in diverse_indices]  # корректируем индексы

        selected_states = torch.stack([hidden_states[i] for i in diverse_indices])
        averaged = torch.mean(selected_states, dim=0)

        mask = inputs['attention_mask'].bool()
        pooled = self._mean_pooling(averaged, mask)

        return pooled.squeeze().numpy()

    def _extract_contrastive_pooling(self, text):
        """Контрастивное объединение: подчеркиваем различия между слоями"""
        hidden_states, _, inputs = self._get_token_embeddings(text)

        # Берем верхние слои
        top_layers = hidden_states[-4:]

        # Вычисляем "контрастивные" веса
        contrasts = []
        for i, layer in enumerate(top_layers):
            if i == 0:
                contrasts.append(layer)
            else:
                # Подчеркиваем различия с предыдущим слоем
                contrast = layer - 0.3 * top_layers[i-1]
                contrasts.append(contrast)

        # Комбинируем контрастивные представления
        final_contrast = torch.mean(torch.stack(contrasts), dim=0)

        mask = inputs['attention_mask'].bool()
        pooled = self._mean_pooling(final_contrast, mask)

        return pooled.squeeze().numpy()

    def _mean_pooling(self, token_embeddings, attention_mask):
        """Усреднение токенов с учетом маски внимания"""
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        pooled = torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

        # Убеждаемся, что возвращаем 2D тензор [batch_size, hidden_dim]
        if pooled.dim() == 1:
            pooled = pooled.unsqueeze(0)
        elif pooled.dim() > 2:
            pooled = pooled.view(pooled.size(0), -1)

        return pooled

    def extract_embeddings_for_method(self, texts, method_name):
        """Извлечение эмбеддингов для всех текстов с выбранным методом"""
        method = self.extraction_methods[method_name]
        embeddings = []

        print(f"Извлечение эмбеддингов методом '{method_name}'...")
        for text in tqdm(texts, desc=f"Обработка {method_name}",
                        leave=True, ncols=100, file=sys.stdout):
            try:
                embedding = method(text)

                # Проверяем и исправляем размерность
                if isinstance(embedding, np.ndarray):
                    if embedding.ndim == 1:
                        embeddings.append(embedding)
                    elif embedding.ndim == 2:
                        # Если 2D, берем первую строку (batch_size=1)
                        embeddings.append(embedding[0])
                    else:
                        # Если больше 2D, сглаживаем
                        embeddings.append(embedding.flatten()[:768])  # обрезаем до нужной размерности
                else:
                    # Если не numpy array, конвертируем
                    embedding = np.array(embedding)
                    if embedding.ndim > 1:
                        embedding = embedding.flatten()[:768]
                    embeddings.append(embedding)

            except Exception as e:
                print(f"Ошибка для текста: {text[:50]}... - {e}")
                # Добавляем нулевой вектор в случае ошибки
                embeddings.append(np.zeros(768))

        # Финальная проверка размерности
        embeddings_array = np.array(embeddings)

        # Если все еще есть проблемы с размерностью, приводим к правильной форме
        if embeddings_array.ndim != 2:
            print(f"Исправление размерности массива: {embeddings_array.shape} -> (?, 768)")
            embeddings_list = []
            for emb in embeddings:
                if isinstance(emb, np.ndarray):
                    if emb.ndim == 0:
                        embeddings_list.append(np.zeros(768))
                    elif emb.ndim == 1:
                        if len(emb) >= 768:
                            embeddings_list.append(emb[:768])
                        else:
                            padded = np.zeros(768)
                            padded[:len(emb)] = emb
                            embeddings_list.append(padded)
                    else:
                        flattened = emb.flatten()
                        if len(flattened) >= 768:
                            embeddings_list.append(flattened[:768])
                        else:
                            padded = np.zeros(768)
                            padded[:len(flattened)] = flattened
                            embeddings_list.append(padded)
                else:
                    embeddings_list.append(np.zeros(768))
            embeddings_array = np.array(embeddings_list)

        print(f"Итоговая размерность эмбеддингов: {embeddings_array.shape}")
        return embeddings_array

    def evaluate_clustering_quality(self, embeddings, labels=None):
        """Оценка качества кластеризации"""

        # Проверяем размерность входных данных
        print(f"Проверка размерности эмбеддингов: {embeddings.shape}")

        if embeddings.ndim != 2:
            print(f"Неправильная размерность эмбеддингов: {embeddings.shape}")
            return {
                'silhouette_score': -1,
                'davies_bouldin_score': float('inf'),
                'calinski_harabasz_score': 0,
                'trustworthiness': 0,
                'n_clusters': 0,
                'noise_ratio': 1.0,
                'umap_embeddings': np.zeros((len(embeddings), 2)),
                'cluster_labels': np.full(len(embeddings), -1)
            }

        # Проверяем на NaN и inf
        if np.any(np.isnan(embeddings)) or np.any(np.isinf(embeddings)):
            print("Найдены NaN или inf значения в эмбеддингах")
            embeddings = np.nan_to_num(embeddings, nan=0.0, posinf=1.0, neginf=-1.0)

        try:
            # Применяем UMAP для снижения размерности
            umap_reducer = umap.UMAP(
                n_components=2,
                n_neighbors=min(20, len(embeddings)-1),  # учитываем размер данных
                min_dist=0.1,
                metric='cosine',
                random_state=42
            )

            umap_embeddings = umap_reducer.fit_transform(embeddings)

            # Кластеризация с HDBSCAN
            min_cluster_size = max(3, min(5, len(embeddings)//10))  # адаптивный размер
            clusterer = HDBSCAN(min_cluster_size=min_cluster_size, metric='euclidean')
            cluster_labels = clusterer.fit_predict(umap_embeddings)

        except Exception as e:
            print(f"Ошибка при UMAP/кластеризации: {e}")
            return {
                'silhouette_score': -1,
                'davies_bouldin_score': float('inf'),
                'calinski_harabasz_score': 0,
                'trustworthiness': 0,
                'n_clusters': 0,
                'noise_ratio': 1.0,
                'umap_embeddings': np.zeros((len(embeddings), 2)),
                'cluster_labels': np.full(len(embeddings), -1)
            }

        # Исключаем шум (-1 метки) для вычисления метрик
        valid_mask = cluster_labels != -1
        if valid_mask.sum() < 2:
            return {
                'silhouette_score': -1,
                'davies_bouldin_score': float('inf'),
                'calinski_harabasz_score': 0,
                'trustworthiness': 0,
                'n_clusters': 0,
                'noise_ratio': 1.0,
                'umap_embeddings': umap_embeddings,
                'cluster_labels': cluster_labels
            }

        valid_embeddings = umap_embeddings[valid_mask]
        valid_labels = cluster_labels[valid_mask]

        # Вычисляем метрики
        try:
            silhouette = silhouette_score(valid_embeddings, valid_labels)
            davies_bouldin = davies_bouldin_score(valid_embeddings, valid_labels)
            calinski_harabasz = calinski_harabasz_score(valid_embeddings, valid_labels)
        except Exception as e:
            print(f"Ошибка при вычислении метрик: {e}")
            silhouette = -1
            davies_bouldin = float('inf')
            calinski_harabasz = 0

        # Вычисляем trustworthiness
        try:
            from sklearn.manifold import trustworthiness as sklearn_trustworthiness
            trustworthiness = sklearn_trustworthiness(embeddings, umap_embeddings, n_neighbors=min(10, len(embeddings)-1))
        except Exception as e:
            print(f"Ошибка при вычислении trustworthiness: {e}")
            trustworthiness = 0

        return {
            'silhouette_score': silhouette,
            'davies_bouldin_score': davies_bouldin,
            'calinski_harabasz_score': calinski_harabasz,
            'trustworthiness': trustworthiness,
            'n_clusters': len(np.unique(valid_labels)),
            'noise_ratio': (cluster_labels == -1).mean(),
            'umap_embeddings': umap_embeddings,
            'cluster_labels': cluster_labels
        }

    def _calculate_trustworthiness(self, X_original, X_embedded, k=10):
        """Вычисление trustworthiness score"""
        from sklearn.neighbors import NearestNeighbors

        # Находим ближайших соседей в исходном и проецированном пространстве
        nn_original = NearestNeighbors(n_neighbors=k+1, metric='cosine').fit(X_original)
        nn_embedded = NearestNeighbors(n_neighbors=k+1, metric='euclidean').fit(X_embedded)

        _, indices_original = nn_original.kneighbors(X_original)
        _, indices_embedded = nn_embedded.kneighbors(X_embedded)

        n = X_original.shape[0]
        trustworthiness = 0

        for i in range(n):
            # Исключаем сам элемент (первый в списке соседей)
            neighbors_original = set(indices_original[i][1:])
            neighbors_embedded = set(indices_embedded[i][1:])

            # Находим "интрудеров" - точки, которые близки в проекции, но далеки в оригинале
            intruders = neighbors_embedded - neighbors_original

            rank_sum = 0
            for intruder in intruders:
                # Находим ранг интрудера в исходном пространстве
                rank = np.where(indices_original[i] == intruder)[0]
                if len(rank) > 0:
                    rank_sum += max(0, rank[0] - k)

            trustworthiness += rank_sum

        # Нормализация
        max_sum = k * (2 * n - 3 * k - 1) / 2
        trustworthiness = 1 - (2 * trustworthiness) / (n * max_sum) if max_sum > 0 else 1

        return max(0, trustworthiness)

    def run_comprehensive_experiment(self, df, text_column='cleaned_text', verb_column='masked_verb'):
        """Запуск полного эксперимента со всеми методами"""
        texts = df[text_column].tolist()
        verbs = df[verb_column].tolist() if verb_column in df.columns else None

        print(f"Начинаем эксперимент с {len(texts)} текстами и {len(self.extraction_methods)} методами")
        print(f"Методы: {list(self.extraction_methods.keys())}")

        results = {}
        embeddings_cache = {}

        # Извлекаем эмбеддинги для каждого метода
        for method_name in tqdm(self.extraction_methods.keys(),
                               desc="Обработка методов", leave=True, ncols=100):
            embeddings = self.extract_embeddings_for_method(texts, method_name)
            embeddings_cache[method_name] = embeddings

            # Оцениваем качество кластеризации
            print(f"Оценка качества для метода '{method_name}'...")
            quality_metrics = self.evaluate_clustering_quality(embeddings, verbs)

            results[method_name] = {
                'embeddings': embeddings,
                'metrics': quality_metrics
            }

            print(f"{method_name}: Silhouette={quality_metrics['silhouette_score']:.3f}, "
                  f"Clusters={quality_metrics['n_clusters']}, "
                  f"Trustworthiness={quality_metrics['trustworthiness']:.3f}")

        return results

    def plot_comparison_results(self, results, save_path=None):
          """
          Визуализация результатов сравнения методов извлечения эмбеддингов.

          Args:
              results (dict): Результаты выполнения экспериментов
              save_path (str, optional): Путь для сохранения графиков

          Returns:
              list: Ранжированный список методов по композитному рейтингу
          """
          # Настройка стиля графиков
          sns.set(style="whitegrid")
          palette = sns.color_palette("husl", 8)
          plt.figure(figsize=(14, 10))


    def plot_comparison_results(self, results, save_path=None):
          """Визуализация результатов сравнения методов"""
          # Подготавливаем данные для сравнения
          methods = list(results.keys())
          metrics_data = {
              'Method': [],
              'Silhouette Score': [],
              'Davies-Bouldin Score': [],
              'Calinski-Harabasz Score': [],
              'Trustworthiness': [],
              'N Clusters': [],
              'Noise Ratio': []
          }

          for method in methods:
              metrics = results[method]['metrics']
              metrics_data['Method'].append(method)
              metrics_data['Silhouette Score'].append(metrics['silhouette_score'])
              metrics_data['Davies-Bouldin Score'].append(metrics['davies_bouldin_score'])
              metrics_data['Calinski-Harabasz Score'].append(metrics['calinski_harabasz_score'])
              metrics_data['Trustworthiness'].append(metrics['trustworthiness'])
              metrics_data['N Clusters'].append(metrics['n_clusters'])
              metrics_data['Noise Ratio'].append(metrics['noise_ratio'])

          # Создаем большую фигуру для всех графиков
          fig, axes = plt.subplots(2, 3, figsize=(18, 12))
          fig.suptitle('Сравнение методов извлечения эмбеддингов UmBERTo', fontsize=16, fontweight='bold')

          # График 1: Silhouette Score
          ax1 = axes[0, 0]
          bars1 = ax1.bar(range(len(methods)), metrics_data['Silhouette Score'],
                        color=palette[:len(methods)])
          ax1.set_title('Silhouette Score (выше = лучше)')
          ax1.set_xlabel('Методы')
          ax1.set_ylabel('Score')
          ax1.set_xticks(range(len(methods)))
          ax1.set_xticklabels(methods, rotation=45, ha='right')
          ax1.grid(True, alpha=0.3)

          # График 2: Davies-Bouldin Score
          ax2 = axes[0, 1]
          bars2 = ax2.bar(range(len(methods)), metrics_data['Davies-Bouldin Score'],
                        color=palette[:len(methods)])
          ax2.set_title('Davies-Bouldin Score (ниже = лучше)')
          ax2.set_xlabel('Методы')
          ax2.set_ylabel('Score')
          ax2.set_xticks(range(len(methods)))
          ax2.set_xticklabels(methods, rotation=45, ha='right')
          ax2.grid(True, alpha=0.3)

          # График 3: Trustworthiness
          ax3 = axes[0, 2]
          bars3 = ax3.bar(range(len(methods)), metrics_data['Trustworthiness'],
                        color=palette[:len(methods)])
          ax3.set_title('Trustworthiness (выше = лучше)')
          ax3.set_xlabel('Методы')
          ax3.set_ylabel('Score')
          ax3.set_xticks(range(len(methods)))
          ax3.set_xticklabels(methods, rotation=45, ha='right')
          ax3.grid(True, alpha=0.3)

          # График 4: Количество кластеров
          ax4 = axes[1, 0]
          bars4 = ax4.bar(range(len(methods)), metrics_data['N Clusters'],
                        color=palette[:len(methods)])
          ax4.set_title('Количество кластеров')
          ax4.set_xlabel('Методы')
          ax4.set_ylabel('Количество')
          ax4.set_xticks(range(len(methods)))
          ax4.set_xticklabels(methods, rotation=45, ha='right')
          ax4.grid(True, alpha=0.3)

          # График 5: Доля шума
          ax5 = axes[1, 1]
          bars5 = ax5.bar(range(len(methods)), metrics_data['Noise Ratio'],
                        color=palette[:len(methods)])
          ax5.set_title('Доля шума (ниже = лучше)')
          ax5.set_xlabel('Методы')
          ax5.set_ylabel('Доля')
          ax5.set_xticks(range(len(methods)))
          ax5.set_xticklabels(methods, rotation=45, ha='right')
          ax5.grid(True, alpha=0.3)

          # График 6: Комбинированный рейтинг
          ax6 = axes[1, 2]
          # Нормализуем метрики и создаем составной рейтинг
          normalized_silhouette = np.array(metrics_data['Silhouette Score'])
          normalized_trustworthiness = np.array(metrics_data['Trustworthiness'])
          normalized_db = 1 / (1 + np.array(metrics_data['Davies-Bouldin Score']))  # инвертируем
          normalized_noise = 1 - np.array(metrics_data['Noise Ratio'])  # инвертируем

          composite_score = (normalized_silhouette + normalized_trustworthiness +
                            normalized_db + normalized_noise) / 4

          bars6 = ax6.bar(range(len(methods)), composite_score,
                        color=palette[:len(methods)])
          ax6.set_title('Композитный рейтинг (выше = лучше)')
          ax6.set_xlabel('Методы')
          ax6.set_ylabel('Score')
          ax6.set_xticks(range(len(methods)))
          ax6.set_xticklabels(methods, rotation=45, ha='right')
          ax6.grid(True, alpha=0.3)

          plt.tight_layout()

          if save_path:
              plt.savefig(save_path, dpi=300, bbox_inches='tight')

          plt.show()

          # Возвращаем рейтинг методов
          ranking = sorted(zip(methods, composite_score), key=lambda x: x[1], reverse=True)
          return ranking

In [ ]:
experiment = RoBERTaEmbeddingExperiment()

# Запуск полного эксперимента
results = experiment.run_comprehensive_experiment(
    en_featured_df[:200],
    text_column='cleaned_text',    # колонка с маскированными текстами
    verb_column='masked_verb'      # колонка с замаскированными глаголами
)

# Визуализация результатов
ranking = experiment.plot_comparison_results(results)

# Вывод лучших методов
print("ТОП-3 метода:")
for i, (method, score) in enumerate(ranking[:3], 1):
    print(f"{i}. {method}: {score:.4f}")

#### Создание эмбеддингов с помощью лучшего метода агрегации

In [ ]:
def generate_dynamic_layer_embeddings(
    texts,
    model,
    tokenizer,
    max_length=512,
    device="cpu",
    batch_size=16,
    show_progress=True,
    n_layers=4
):
    """
    Генерирует эмбеддинги с динамическим выбором слоев

    Параметры:
        texts (list): Список текстов для обработки
        model: Предварительно загруженная модель
        tokenizer: Предварительно загруженный токенизатор
        max_length (int): Максимальная длина текста
        device (str): Устройство для вычислений (cpu/cuda)
        batch_size (int): Размер пакета обработки
        show_progress (bool): Показывать прогресс-бар
        n_layers (int): Количество слоев для выбора
    """
    # Валидация входных данных
    if not texts:
        return np.array([])

    # Подготовка прогресс-бара
    total_batches = (len(texts) + batch_size - 1) // batch_size
    pbar = tqdm(total=total_batches, desc="Generating embeddings", disable=not show_progress)

    all_embeddings = []

    # Обработка батчами
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i+batch_size]

        # Токенизация батча
        inputs = tokenizer(
            batch_texts,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=max_length
        ).to(device)

        # Получение скрытых состояний
        with torch.no_grad():
            outputs = model(**inputs)
            hidden_states = outputs.hidden_states

        # Вычисление косинусной схожести между слоями
        layer_similarities = []
        for i in range(6, len(hidden_states)-1):
            state1 = hidden_states[i].mean(dim=1)
            state2 = hidden_states[i+1].mean(dim=1)
            similarity = torch.cosine_similarity(state1, state2, dim=-1)
            layer_similarities.append(similarity.mean().item())

        # Выбор наиболее различных слоев
        similarity_scores = np.array(layer_similarities)
        diverse_indices = np.argsort(similarity_scores)[:n_layers] + 6

        # Усреднение выбранных слоев
        selected_states = torch.stack([hidden_states[idx] for idx in diverse_indices])
        averaged = torch.mean(selected_states, dim=0)

        # Пулинг с маской внимания
        mask = inputs['attention_mask'].unsqueeze(-1).float()
        summed = torch.sum(averaged * mask, dim=1)
        counts = torch.clamp(mask.sum(dim=1), min=1e-9)
        batch_embeddings = (summed / counts).cpu().numpy()

        all_embeddings.append(batch_embeddings)
        pbar.update(1)

    pbar.close()
    return np.vstack(all_embeddings)

In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

# Загрузка модели и токенизатора
model_name = "sentence-transformers/all-roberta-large-v1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForMaskedLM.from_pretrained(model_name, output_hidden_states=True)
model.eval()

In [ ]:
# Генерация эмбеддингов
embeddings = generate_dynamic_layer_embeddings(
    texts=en_featured_df['cleaned_text'].tolist(),
    model=model,
    tokenizer=tokenizer,
    max_length=512,
    device="cpu",
    batch_size=16
)

print(f"Размерность итоговых эмбеддингов: {embeddings.shape}")
print(f"Пример эмбеддинга: {embeddings[0][:10]}")

In [ ]:
import pickle

with open("/content/drive/MyDrive/Диплом бакалавриат'25/english_embeddings.pkl", 'wb') as f:
    pickle.dump(embeddings, f)

In [22]:
with open("/content/drive/MyDrive/Диплом бакалавриат'25/english_embeddings.pkl", 'rb') as f:
    embeddings = pickle.load(f)

In [23]:
embeddings_np = np.array(embeddings)

In [ ]:
# Проверка нормализации
norms = np.linalg.norm(embeddings_np, axis=1)
print("Минимальная норма:", np.min(norms))
print("Максимальная норма:", np.max(norms))

In [ ]:
# Проверка близости к 1
tolerance = 1e-5
is_normalized = np.allclose(norms, 1.0, atol=tolerance)
print(f"Все эмбеддинги нормализованы (с погрешностью {tolerance})? {is_normalized}")

In [26]:
embeddings_l2 = normalize(embeddings, norm='l2')

#### Снижение размерности

In [ ]:
param_grid = {
    'n_neighbors': np.arange(10, 50, 10),
    'min_dist': [0.01, 0.05],
    'n_components': np.arange(2, 30, 5),
    'metric': ['cosine', 'correlation']
}

In [ ]:
def bootstrap_stability(embeddings, params, n_samples=5, sample_size=0.8):
    stability = []
    for _ in range(n_samples):
        idx = np.random.choice(len(embeddings), int(len(embeddings)*sample_size))
        proj1 = umap.UMAP(**params).fit_transform(embeddings[idx])
        proj2 = umap.UMAP(**params).fit_transform(embeddings[idx])
        stability.append(spearmanr(proj1.flatten(), proj2.flatten())[0])
    return np.mean(stability)

In [ ]:
def optimize_umap_params(embeddings, param_grid, sample_size=1000, random_state=42):
    """
    Оптимизирует параметры UMAP для заданных эмбеддингов с использованием грид-поиска.

    Параметры:
    embeddings (np.array/torch.Tensor): Массив эмбеддингов
    param_grid (dict): Сетка параметров для поиска
    sample_size (int): Размер выборки для ускорения вычислений
    random_state (int): Seed для воспроизводимости

    Возвращает:
    pd.DataFrame: Результаты экспериментов с метриками
    """
    # Сэмплирование данных
    embeddings = embeddings[:sample_size]

    # Стандартизация
    scaler = StandardScaler()
    embeddings_scaled = scaler.fit_transform(embeddings)

    # Основной цикл оптимизации
    results = []
    for params in tqdm(ParameterGrid(param_grid), desc="Grid Search"):
        reducer = umap.UMAP(**params, random_state=random_state)
        projection = reducer.fit_transform(embeddings_scaled)

        # Вычисление метрик
        d_orig = pdist(embeddings_scaled, metric=params['metric'])
        d_proj = pdist(projection, metric='euclidean')

        metrics = {
            'trustworthiness': trustworthiness(embeddings_scaled, projection,
                                             n_neighbors=params['n_neighbors']),
            'stability': bootstrap_stability(embeddings_scaled, params),
            'global_spearman': spearmanr(d_orig, d_proj).correlation
        }

        results.append({**params, **metrics})

    return pd.DataFrame(results)


In [ ]:
results_df = optimize_umap_params(
    embeddings=embeddings_l2,
    param_grid=param_grid,
    sample_size=460
    )

Grid Search: 100%|██████████| 96/96 [23:08<00:00, 14.46s/it]


In [ ]:
def find_best_parameters(results_df, metric_weights=None):
    """
    Находит оптимальные параметры на основе взвешенной комбинации метрик.

    Параметры:
    results_df (pd.DataFrame): Результаты экспериментов
    metric_weights (dict): Веса для метрик (по умолчанию равные веса)

    Возвращает:
    pd.DataFrame: Топ-5 лучших комбинаций с нормализованными оценками
    """
    df = results_df.copy()

    # Задаем веса по умолчанию (равные веса)
    if not metric_weights:
        metric_weights = {
            'trustworthiness': 0.5,
            'stability': 0.3,
            'global_spearman': 0.2
        }

    # Нормализуем метрики (Min-Max scaling)
    for metric in metric_weights:
        df[metric + '_norm'] = (df[metric] - df[metric].min()) / (df[metric].max() - df[metric].min())

    # Рассчитываем общий score
    df['total_score'] = sum(
        df[metric + '_norm'] * weight
        for metric, weight in metric_weights.items()
    )

    # Сортируем по убыванию общего score
    best_results = df.sort_values('total_score', ascending=False).head(5)

    return best_results

best_df = find_best_parameters(results_df)

best_df[['n_neighbors', 'min_dist', 'n_components', 'metric',
               'trustworthiness', 'stability', 'global_spearman', 'total_score']]

,n_neighbors,min_dist,n_components,metric,trustworthiness,stability,global_spearman,total_score
92,10,0.05,27,correlation,0.865830,0.848219,0.445784,0.897042
44,10,0.05,27,cosine,0.863157,0.859916,0.442754,0.883249
64,10,0.01,22,correlation,0.864753,0.830985,0.443422,0.880288
88,10,0.05,22,correlation,0.865923,0.780009,0.449904,0.874859
76,10,0.05,7,correlation,0.860728,0.853013,0.446975,0.874245


In [ ]:
param_grid = {
    'n_neighbors': [10],
    'min_dist': [0.05],
    'n_components': [27],
    'metric': ['correlation']
}

In [ ]:
results_df_1 = optimize_umap_params(
    embeddings=embeddings_l2,
    param_grid=param_grid,
    sample_size=len(embeddings_np)
    )

Grid Search: 100%|██████████| 1/1 [09:51<00:00, 591.07s/it]


In [ ]:
param_grid = {
    'n_neighbors': [10],
    'min_dist': [0.05],
    'n_components': [27],
    'metric': ['cosine']
}

In [ ]:
results_df_2 = optimize_umap_params(
    embeddings=embeddings_l2,
    param_grid=param_grid,
    sample_size=len(embeddings_np)
    )

Grid Search: 100%|██████████| 1/1 [06:53<00:00, 413.93s/it]


In [ ]:
param_grid = {
    'n_neighbors': [10],
    'min_dist': [0.01],
    'n_components': [22],
    'metric': ['correlation']
}

In [ ]:
results_df_3 = optimize_umap_params(
    embeddings=embeddings_l2,
    param_grid=param_grid,
    sample_size=len(embeddings_np)
    )

Grid Search: 100%|██████████| 1/1 [08:59<00:00, 539.64s/it]


In [ ]:
results_best = pd.concat([results_df_1, results_df_2, results_df_3], axis=0)
results_best

,metric,min_dist,n_components,n_neighbors,trustworthiness,stability,global_spearman
0,correlation,0.05,27,10,0.902189,0.996618,0.354140
0,cosine,0.05,27,10,0.902855,0.927906,0.353618
0,correlation,0.01,22,10,0.902557,0.754340,0.343341


In [27]:
scaler = StandardScaler()
scaled_embeds = scaler.fit_transform(embeddings_l2)

In [28]:
reducer = umap.UMAP(
    n_components=27,
    n_neighbors=10,
    min_dist=0.05,
    metric='correlation',
    random_state=42
)

In [29]:
umap_embeds = reducer.fit_transform(scaled_embeds)

#### Кластеризация контекстов

In [31]:
class MetaphorClusteringEnsemble:
    """
    Консенсусная кластеризация для выявления метафорических употреблений глаголов
    """

    def __init__(self, data, algorithm_weights=None):
        """
        Инициализация класса

        Parameters:
        -----------
        data : array-like
            Матрица признаков (векторные представления употреблений)
        algorithm_weights : dict
            Веса для каждого алгоритма в консенсусной матрице
        """
        self.data = np.array(data)
        self.n_samples = self.data.shape[0]

        # Веса алгоритмов (HDBSCAN как самый надежный)
        self.weights = algorithm_weights or {
            'hdbscan': 0.5,
            'dbscan': 0.3,
            'spectral': 0.2
        }

        self.best_params = {}
        self.clusterings = {}
        self.consensus_matrix = None
        self.final_clusters = None

    def optimize_dbscan(self, eps_range=None, min_samples_range=None):
        """
        Подбор оптимальных параметров для DBSCAN
        """
        print("Оптимизация параметров DBSCAN...")

        if eps_range is None:
            # Автоматическое определение диапазона eps на основе k-distance
            k = 4  # минимальное количество точек для формирования кластера
            nbrs = NearestNeighbors(n_neighbors=k).fit(self.data)
            distances, indices = nbrs.kneighbors(self.data)
            distances = np.sort(distances[:, k-1], axis=0)

            # Используем метод "локтя" для определения оптимального eps
            knee_point = self._find_knee_point(distances)
            eps_range = np.linspace(distances[knee_point] * 0.5, distances[knee_point] * 2, 20)

        if min_samples_range is None:
            min_samples_range = range(3, min(20, self.n_samples // 10))

        best_score = -1
        best_params = {}

        param_grid = ParameterGrid({
            'eps': eps_range,
            'min_samples': min_samples_range
        })

        for params in param_grid:
            try:
                dbscan = DBSCAN(**params)
                labels = dbscan.fit_predict(self.data)

                # Проверяем, что есть хотя бы 2 кластера (исключая шум)
                n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
                if n_clusters < 2:
                    continue

                # Оценка качества (без учета точек шума для silhouette)
                mask = labels != -1
                if np.sum(mask) < 2:
                    continue

                score = silhouette_score(self.data[mask], labels[mask])

                if score > best_score:
                    best_score = score
                    best_params = params.copy()

            except:
                continue

        self.best_params['dbscan'] = best_params
        print(f"Лучшие параметры DBSCAN: {best_params}, Score: {best_score:.3f}")

    def optimize_hdbscan(self, min_cluster_size_range=None, min_samples_range=None):
        """
        Подбор оптимальных параметров для HDBSCAN
        """
        print("Оптимизация параметров HDBSCAN...")

        if min_cluster_size_range is None:
            min_cluster_size_range = np.arange(3, 50, 7)

        if min_samples_range is None:
            min_samples_range = np.arange(1, 20, 5)

        best_score = -1
        best_params = {}

        param_grid = ParameterGrid({
            'min_cluster_size': min_cluster_size_range,
            'min_samples': min_samples_range,
            'cluster_selection_epsilon': [0.0, 0.1, 0.2, 0.5]
        })

        for params in param_grid:
            try:
                clusterer = hdbscan.HDBSCAN(**params)
                labels = clusterer.fit_predict(self.data)

                n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
                if n_clusters < 2:
                    continue

                mask = labels != -1
                if np.sum(mask) < 2:
                    continue

                # Комбинированная метрика: silhouette + DBCV (если доступно)
                silhouette = silhouette_score(self.data[mask], labels[mask])

                # HDBSCAN предоставляет собственную метрику качества
                try:
                    dbcv_score = clusterer.relative_validity_
                    if dbcv_score is not None:
                        score = 0.4 * silhouette + 0.6 * dbcv_score
                    else:
                        score = silhouette
                except:
                    score = silhouette

                if score > best_score:
                    best_score = score
                    best_params = params.copy()

            except:
                continue

        self.best_params['hdbscan'] = best_params
        print(f"Лучшие параметры HDBSCAN: {best_params}, Score: {best_score:.3f}")

    def optimize_spectral(self, max_clusters=None):
        """
        Подбор оптимального количества кластеров для Spectral Clustering
        """
        print("Оптимизация параметров Spectral Clustering...")

        if max_clusters is None:
            max_clusters = min(20, self.n_samples // 3)

        # Анализ собственных значений лапласиана для определения количества кластеров
        similarity_matrix = self._compute_similarity_matrix()
        L = laplacian(similarity_matrix, normed=True)
        eigenvals = np.linalg.eigvals(L.toarray() if hasattr(L, 'toarray') else L)
        eigenvals = np.sort(eigenvals)

        # Поиск наибольшего разрыва в собственных значениях
        gaps = np.diff(eigenvals[:max_clusters])
        spectral_suggestion = np.argmax(gaps) + 2

        best_score = -1
        best_params = {}

        # Тестируем диапазон кластеров вокруг предложенного спектральным анализом
        k_range = range(max(2, spectral_suggestion - 3),
                       min(max_clusters + 1, spectral_suggestion + 5))

        for n_clusters in k_range:
            try:
                for affinity in ['rbf', 'nearest_neighbors']:
                    spectral = SpectralClustering(
                        n_clusters=n_clusters,
                        affinity=affinity,
                        random_state=42
                    )
                    labels = spectral.fit_predict(self.data)

                    score = silhouette_score(self.data, labels)

                    if score > best_score:
                        best_score = score
                        best_params = {
                            'n_clusters': n_clusters,
                            'affinity': affinity
                        }

            except:
                continue

        self.best_params['spectral'] = best_params
        print(f"Лучшие параметры Spectral: {best_params}, Score: {best_score:.3f}")

    def _compute_similarity_matrix(self):
        """Вычисление матрицы сходства для спектральной кластеризации"""
        distances = pdist(self.data, metric='euclidean')
        distance_matrix = squareform(distances)

        # RBF kernel
        gamma = 1.0 / (2 * np.var(distances))
        similarity_matrix = np.exp(-gamma * distance_matrix ** 2)

        return similarity_matrix

    def _find_knee_point(self, values):
        """Поиск точки 'колена' в отсортированном массиве"""
        n_points = len(values)
        all_coords = np.vstack((range(n_points), values)).T

        # Первая и последняя точки
        first_point = all_coords[0]
        last_point = all_coords[-1]

        # Вектор от первой к последней точке
        line_vec = last_point - first_point
        line_vec_norm = line_vec / np.sqrt(np.sum(line_vec**2))

        # Векторы от первой точки к каждой точке
        vec_from_first = all_coords - first_point

        # Проекции на линию
        scalar_proj = np.dot(vec_from_first, line_vec_norm.reshape(-1, 1)).flatten()
        vec_proj = np.outer(scalar_proj, line_vec_norm)

        # Перпендикулярные расстояния
        distances = np.sqrt(np.sum((vec_from_first - vec_proj)**2, axis=1))

        return np.argmax(distances)

    def fit_all_algorithms(self):
        """Обучение всех алгоритмов с оптимальными параметрами"""
        print("\nОбучение алгоритмов кластеризации...")

        # DBSCAN
        if 'dbscan' in self.best_params:
            dbscan = DBSCAN(**self.best_params['dbscan'])
            self.clusterings['dbscan'] = dbscan.fit_predict(self.data)
            print(f"DBSCAN: {len(set(self.clusterings['dbscan'])) - (1 if -1 in self.clusterings['dbscan'] else 0)} кластеров")

        # HDBSCAN
        if 'hdbscan' in self.best_params:
            hdbscan_clusterer = hdbscan.HDBSCAN(**self.best_params['hdbscan'])
            self.clusterings['hdbscan'] = hdbscan_clusterer.fit_predict(self.data)
            print(f"HDBSCAN: {len(set(self.clusterings['hdbscan'])) - (1 if -1 in self.clusterings['hdbscan'] else 0)} кластеров")

        # Spectral Clustering
        if 'spectral' in self.best_params:
            spectral = SpectralClustering(**self.best_params['spectral'], random_state=42)
            self.clusterings['spectral'] = spectral.fit_predict(self.data)
            print(f"Spectral: {self.best_params['spectral']['n_clusters']} кластеров")

    def compute_consensus_matrix(self):
        """Вычисление взвешенной консенсусной матрицы"""
        print("\nВычисление консенсусной матрицы...")

        self.consensus_matrix = np.zeros((self.n_samples, self.n_samples))
        total_weight = 0

        for alg_name, labels in self.clusterings.items():
            if alg_name not in self.weights:
                continue

            weight = self.weights[alg_name]
            total_weight += weight

            # Создание матрицы совместного вхождения для текущего алгоритма
            cooccurrence_matrix = np.zeros((self.n_samples, self.n_samples))

            for i in range(self.n_samples):
                for j in range(i, self.n_samples):
                    # Проверяем, что обе точки не являются шумом (-1)
                    if labels[i] != -1 and labels[j] != -1 and labels[i] == labels[j]:
                        cooccurrence_matrix[i, j] = 1
                        cooccurrence_matrix[j, i] = 1

            self.consensus_matrix += weight * cooccurrence_matrix

        # Нормализация на общий вес
        self.consensus_matrix /= total_weight

        print("Консенсусная матрица вычислена")

    def extract_final_clusters(self, threshold=0.5):
        """
        Извлечение финальных консенсусных кластеров

        Parameters:
        -----------
        threshold : float
            Порог для определения принадлежности к одному кластеру
        """
        print(f"\nИзвлечение финальных кластеров (порог: {threshold})...")

        # Создание графа связей на основе консенсусной матрицы
        adjacency_matrix = (self.consensus_matrix >= threshold).astype(int)

        # Поиск связанных компонент (кластеров)
        visited = np.zeros(self.n_samples, dtype=bool)
        clusters = []
        cluster_id = 0
        self.final_clusters = np.full(self.n_samples, -1)  # -1 для неназначенных точек

        def dfs(node, current_cluster):
            """Поиск в глубину для нахождения связанных компонент"""
            visited[node] = True
            current_cluster.append(node)

            for neighbor in range(self.n_samples):
                if adjacency_matrix[node, neighbor] and not visited[neighbor]:
                    dfs(neighbor, current_cluster)

        for i in range(self.n_samples):
            if not visited[i]:
                current_cluster = []
                dfs(i, current_cluster)

                # Сохраняем только кластеры с более чем одной точкой
                if len(current_cluster) > 1:
                    clusters.append(current_cluster)
                    for point in current_cluster:
                        self.final_clusters[point] = cluster_id
                    cluster_id += 1

        n_clusters = len(clusters)
        n_noise = np.sum(self.final_clusters == -1)

        print(f"Найдено {n_clusters} консенсусных кластеров")
        print(f"Точек шума: {n_noise}")

        # Статистика по размерам кластеров
        if n_clusters > 0:
            cluster_sizes = [len(cluster) for cluster in clusters]
            print(f"Размеры кластеров: мин={min(cluster_sizes)}, макс={max(cluster_sizes)}, средний={np.mean(cluster_sizes):.1f}")

        return self.final_clusters

    def fit(self, dbscan_params=None, hdbscan_params=None, spectral_params=None,
            consensus_threshold=0.5):
        """
        Полный пайплайн кластеризации

        Parameters:
        -----------
        dbscan_params, hdbscan_params, spectral_params : dict
            Параметры для оптимизации каждого алгоритма
        consensus_threshold : float
            Порог для извлечения финальных кластеров
        """
        print("=== ЗАПУСК КОНСЕНСУСНОЙ КЛАСТЕРИЗАЦИИ ===")

        # Оптимизация параметров
        self.optimize_dbscan(**(dbscan_params or {}))
        self.optimize_hdbscan(**(hdbscan_params or {}))
        self.optimize_spectral(**(spectral_params or {}))

        # Обучение алгоритмов
        self.fit_all_algorithms()

        # Построение консенсуса
        self.compute_consensus_matrix()

        # Извлечение финальных кластеров
        final_clusters = self.extract_final_clusters(threshold=consensus_threshold)

        return final_clusters

In [ ]:
def visualize_consensus_clusters(data, clusters, method='both', figsize=(16, 7),
                               save_path=None, dpi=300):
    """
    Визуализация результатов консенсусной кластеризации

    Parameters:
    -----------
    data : np.ndarray
        Исходные данные (векторные представления)
    clusters : np.ndarray
        Метки кластеров (-1 для шума)
    method : str
        Метод снижения размерности: 'pca', 'tsne', или 'both'
    figsize : tuple
        Размер фигуры
    save_path : str
        Путь для сохранения графика
    dpi : int
        Разрешение для сохранения
    """

    # Настройка стиля и цветов
    sns.set(style="whitegrid", font_scale=1.0)
    palette = sns.color_palette("husl", 8)
    ITALIAN_COLOR = palette[0]

    # Получаем уникальные кластеры (исключаем шум -1)
    unique_clusters = np.unique(clusters)
    noise_mask = clusters == -1
    n_clusters = len(unique_clusters) - (1 if -1 in unique_clusters else 0)

    # Создаем цветовую палитру
    cluster_colors = sns.color_palette("husl", n_colors=max(n_clusters, 8))
    if n_clusters > 0:
        cluster_colors[0] = ITALIAN_COLOR  # Первый кластер всегда итальянский

    # Настройка параметров визуализации
    POINT_ALPHA = 0.85
    NOISE_ALPHA = 0.5
    BORDER_ALPHA = 0.9
    ANNOTATION_FONT = {'size': 12, 'weight': 'semibold', 'color': 'white'}

    # Создаем фигуру
    if method == 'both':
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=figsize, facecolor='white')
        axes = [ax1, ax2]
        methods = ['PCA', 't-SNE']
    else:
        fig, ax = plt.subplots(figsize=(figsize[0]//2, figsize[1]), facecolor='white')
        axes = [ax]
        methods = ['PCA' if method == 'pca' else 't-SNE']

    # Применяем методы снижения размерности
    reducers = []
    if 'PCA' in methods:
        pca = PCA(n_components=2, random_state=42)
        data_pca = pca.fit_transform(data)
        reducers.append(('PCA', data_pca, pca))

    if 't-SNE' in methods:
        tsne = TSNE(n_components=2, random_state=42,
                   perplexity=min(30, len(data)//4), init='pca')
        data_tsne = tsne.fit_transform(data)
        reducers.append(('t-SNE', data_tsne, None))

    # Создаем визуализации
    for idx, (reduction_name, data_2d, reducer) in enumerate(reducers):
        ax = axes[idx]
        ax.set_facecolor('white')

        # Рисуем шум (если есть)
        if np.any(noise_mask):
            ax.scatter(data_2d[noise_mask, 0], data_2d[noise_mask, 1],
                      c='#d9d9d9', alpha=NOISE_ALPHA, s=25,
                      marker='.', edgecolors='none',
                      label='Шум')

        # Визуализация кластеров
        cluster_counter = 1  # Начинаем нумерацию с 1
        for cluster_id in unique_clusters:
            if cluster_id == -1: continue

            mask = clusters == cluster_id
            cluster_points = data_2d[mask]
            color = cluster_colors[(cluster_counter-1) % len(cluster_colors)]

            # Основные точки кластера
            ax.scatter(cluster_points[:, 0], cluster_points[:, 1],
                     c=[color], alpha=POINT_ALPHA, s=65,
                     edgecolors='w', linewidth=0.6,
                     label=f'Кластер {cluster_counter}', zorder=3)

            # Границы кластера
            if len(cluster_points) >= 3:
                try:
                    hull = ConvexHull(cluster_points)
                    hull_points = cluster_points[hull.vertices]

                    # Сглаженные границы
                    poly = mpatches.Polygon(hull_points, closed=True,
                                          fill=False, edgecolor=color,
                                          linewidth=1.8, linestyle='-',
                                          alpha=BORDER_ALPHA, zorder=2)
                    ax.add_patch(poly)
                except:
                    if len(cluster_points) >= 2:
                        cov = np.cov(cluster_points.T)
                        vals, vecs = np.linalg.eigh(cov)
                        angle = np.degrees(np.arctan2(*vecs[:, 0][::-1]))
                        w, h = 3 * np.sqrt(vals)

                        ellipse = Ellipse(cluster_points.mean(0), w, h, angle,
                                         fill=False, edgecolor=color,
                                         linewidth=1.8, alpha=BORDER_ALPHA)
                        ax.add_patch(ellipse)

            # Аннотация кластера
            center = cluster_points.mean(axis=0)
            ax.text(center[0], center[1], str(cluster_counter),
                   ha='center', va='center', **ANNOTATION_FONT,
                   bbox=dict(boxstyle="circle,pad=0.3",
                            facecolor=color, alpha=0.9,
                            edgecolor='none'))

            cluster_counter += 1

        # Заголовки и подзаголовки
        ax.set_title(f'Консенсусные кластеры\n(Метод проекции: {reduction_name})',
                    fontsize=13, pad=12)

        # Оформление осей
        ax.set_xlabel(f'{reduction_name} 1', labelpad=10)
        ax.set_ylabel(f'{reduction_name} 2', labelpad=10)
        ax.tick_params(axis='both', which='both', length=0)

        # Добавление легенды
        if idx == 0:
            handles = [plt.Line2D([0], [0], marker='o', color='w',
                      markerfacecolor=c, markersize=10, alpha=0.8)
                     for i, c in enumerate(cluster_colors) if i < n_clusters]
            labels = [f'Кластер {i+1}' for i in range(n_clusters)]
            if np.any(noise_mask):
                handles.append(plt.Line2D([0], [0], marker='.', color='w',
                              markerfacecolor='#d9d9d9', markersize=10))
                labels.append('Шум')

            legend = ax.legend(handles, labels, loc='upper right',
                             bbox_to_anchor=(1.15, 1.02),
                             frameon=True, framealpha=0.95,
                             borderpad=1)
            legend.get_frame().set_edgecolor('#333333')

        # Объясненная дисперсия для PCA
        if reduction_name == 'PCA' and reducer is not None:
            explained_var = reducer.explained_variance_ratio_
            ax.text(0.03, 0.97,
                   f'Expl. Variance:\nPC1: {explained_var[0]:.1%}\nPC2: {explained_var[1]:.1%}',
                   transform=ax.transAxes, ha='left', va='top',
                   fontsize=10, bbox=dict(facecolor='white', alpha=0.8,
                                       edgecolor='none', pad=6))

    # Общий заголовок
    if method == 'both':
        plt.suptitle('Результаты консенсусной кластеризации',
                   fontsize=16, y=1.02, weight='bold')

    # Финализация
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])

    if save_path:
        plt.savefig(save_path, dpi=dpi, bbox_inches='tight',
                   facecolor='white', transparent=False)
        print(f"График сохранён: {save_path}")

    plt.show()

    # Вывод статистики
    print("\n=== СТАТИСТИКА ===")
    print(f"Кластеров: {n_clusters}")
    print(f"Шум: {noise_mask.sum()} точек")
    print(f"Всего данных: {len(data)}")

    if n_clusters > 0:
        sizes = [np.sum(clusters == c) for c in unique_clusters if c != -1]
        print(f"\nРазмеры кластеров:")
        print(pd.Series(sizes).describe().to_string())

In [ ]:
def analyze_cluster_medoids(data, clusters, contexts=None, top_k=20,
                           distance_metric='euclidean', return_indices=False):
    """
    Анализ медоидов кластеров и поиск ближайших к ним контекстов

    Parameters:
    -----------
    data : np.ndarray
        Матрица признаков (векторные представления)
    clusters : np.ndarray
        Метки кластеров для каждой точки (-1 для шума)
    contexts : list or pd.Series, optional
        Список контекстов/предложений соответствующих каждой точке данных
    top_k : int, default=20
        Количество ближайших контекстов для вывода
    distance_metric : str, default='euclidean'
        Метрика расстояния для поиска медоида
    return_indices : bool, default=False
        Возвращать ли индексы медоидов и ближайших точек

    Returns:
    --------
    dict : Словарь с результатами анализа для каждого кластера
    """

    # Проверяем входные данные
    if len(data) != len(clusters):
        raise ValueError("Размеры data и clusters должны совпадать")

    if contexts is not None and len(contexts) != len(data):
        raise ValueError("Размеры contexts и data должны совпадать")

    # Получаем уникальные кластеры (исключаем шум -1)
    unique_clusters = np.unique(clusters)
    valid_clusters = unique_clusters[unique_clusters != -1]

    results = {}

    print("=== АНАЛИЗ МЕДОИДОВ КЛАСТЕРОВ ===\n")

    for cluster_id in valid_clusters:
        print(f" КЛАСТЕР {cluster_id}")
        print("=" * 50)

        # Получаем точки текущего кластера
        cluster_mask = clusters == cluster_id
        cluster_data = data[cluster_mask]
        cluster_indices = np.where(cluster_mask)[0]
        cluster_size = len(cluster_data)

        print(f"Размер кластера: {cluster_size} точек")

        # Находим медоид - точку с минимальной суммой расстояний до всех остальных в кластере
        if cluster_size == 1:
            medoid_idx_in_cluster = 0
            medoid_distance_sum = 0.0
        else:
            # Вычисляем матрицу расстояний внутри кластера
            intra_distances = pairwise_distances(cluster_data, metric=distance_metric)

            # Находим точку с минимальной суммой расстояний (медоид)
            distance_sums = np.sum(intra_distances, axis=1)
            medoid_idx_in_cluster = np.argmin(distance_sums)
            medoid_distance_sum = distance_sums[medoid_idx_in_cluster]

        # Индекс медоида в исходных данных
        medoid_global_idx = cluster_indices[medoid_idx_in_cluster]
        medoid_vector = data[medoid_global_idx]

        print(f"Медоид: точка #{medoid_global_idx}")
        print(f"Средняя внутрикластерная дистанция медоида: {medoid_distance_sum / cluster_size:.4f}")

        # Показываем контекст медоида, если есть
        if contexts is not None:
            print(f"\n КОНТЕКСТ МЕДОИДА:")
            print(f"   \"{contexts[medoid_global_idx]}\"\n")

        # Находим top_k ближайших точек к медоиду из всего датасета
        distances_to_medoid = cdist([medoid_vector], data, metric=distance_metric)[0]

        # Сортируем по расстоянию и берем top_k
        nearest_indices = np.argsort(distances_to_medoid)[:top_k]
        nearest_distances = distances_to_medoid[nearest_indices]

        print(f"ТОП-{top_k} БЛИЖАЙШИХ КОНТЕКСТОВ К МЕДОИДУ:")
        print("-" * 60)

        # Сохраняем результаты для кластера
        cluster_results = {
            'medoid_index': medoid_global_idx,
            'medoid_vector': medoid_vector,
            'cluster_size': cluster_size,
            'avg_intra_distance': medoid_distance_sum / cluster_size,
            'nearest_indices': nearest_indices,
            'nearest_distances': nearest_distances
        }

        if contexts is not None:
            cluster_results['medoid_context'] = contexts[medoid_global_idx]
            cluster_results['nearest_contexts'] = [contexts[idx] for idx in nearest_indices]

            # Выводим ближайшие контексты
            for rank, (idx, dist) in enumerate(zip(nearest_indices, nearest_distances), 1):
                # Отмечаем, к какому кластеру принадлежит каждая точка
                point_cluster = clusters[idx]
                cluster_marker = f"[К{point_cluster}]" if point_cluster != -1 else "[ШУМ]"

                # Особо выделяем медоид
                is_medoid = "МЕДОИД" if idx == medoid_global_idx else ""

                print(f"{rank:2d}. {cluster_marker} (дист: {dist:.4f}) {is_medoid}")
                print(f"    \"{contexts[idx]}\"")
                print()
        else:
            # Если контекстов нет, просто показываем индексы и расстояния
            for rank, (idx, dist) in enumerate(zip(nearest_indices, nearest_distances), 1):
                point_cluster = clusters[idx]
                cluster_marker = f"[К{point_cluster}]" if point_cluster != -1 else "[ШУМ]"
                is_medoid = "МЕДОИД" if idx == medoid_global_idx else ""

                print(f"{rank:2d}. Точка #{idx} {cluster_marker} (дист: {dist:.4f}) {is_medoid}")

        results[cluster_id] = cluster_results
        print("\n" + "="*80 + "\n")

    # Выводим общую статистику
    print("ОБЩАЯ СТАТИСТИКА ПО КЛАСТЕРАМ")
    print("=" * 50)

    for cluster_id, result in results.items():
        print(f"Кластер {cluster_id}: {result['cluster_size']} точек, "
              f"средняя дистанция до медоида: {result['avg_intra_distance']:.4f}")

    if return_indices:
        return results
    else:
        # Возвращаем упрощенную версию без векторов для удобства
        simple_results = {}
        for cluster_id, result in results.items():
            simple_results[cluster_id] = {
                'medoid_index': result['medoid_index'],
                'cluster_size': result['cluster_size'],
                'avg_intra_distance': result['avg_intra_distance'],
                'nearest_indices': result['nearest_indices'].tolist(),
                'nearest_distances': result['nearest_distances'].tolist()
            }
            if contexts is not None:
                simple_results[cluster_id]['medoid_context'] = result['medoid_context']
                simple_results[cluster_id]['nearest_contexts'] = result['nearest_contexts']

        return simple_results

def compare_medoids_similarity(results, data, distance_metric='euclidean'):
    """
    Дополнительная функция для анализа схожести между медоидами разных кластеров

    Parameters:
    -----------
    results : dict
        Результаты функции analyze_cluster_medoids с return_indices=True
    data : np.ndarray
        Исходная матрица данных
    distance_metric : str
        Метрика для вычисления расстояний
    """

    if len(results) < 2:
        print("Недостаточно кластеров для сравнения медоидов")
        return

    print("\n МАТРИЦА РАССТОЯНИЙ МЕЖДУ МЕДОИДАМИ")
    print("=" * 60)

    cluster_ids = list(results.keys())
    medoid_vectors = np.array([results[cid]['medoid_vector'] for cid in cluster_ids])

    # Вычисляем матрицу расстояний между медоидами
    medoid_distances = pairwise_distances(medoid_vectors, metric=distance_metric)

    # Создаем красивую таблицу
    df_distances = pd.DataFrame(medoid_distances,
                               index=[f"К{cid}" for cid in cluster_ids],
                               columns=[f"К{cid}" for cid in cluster_ids])

    print(df_distances.round(4))

    # Находим самые близкие и далекие пары
    mask = np.triu(np.ones_like(medoid_distances, dtype=bool), k=1)
    upper_distances = medoid_distances[mask]

    if len(upper_distances) > 0:
        min_dist_idx = np.unravel_index(np.argmin(medoid_distances + np.eye(len(cluster_ids)) * 1000),
                                       medoid_distances.shape)
        max_dist_idx = np.unravel_index(np.argmax(medoid_distances), medoid_distances.shape)

        print(f"Самые близкие кластеры: К{cluster_ids[min_dist_idx[0]]} ↔ К{cluster_ids[min_dist_idx[1]]} "
              f"(расстояние: {medoid_distances[min_dist_idx]:.4f})")
        print(f"Самые далекие кластеры: К{cluster_ids[max_dist_idx[0]]} ↔ К{cluster_ids[max_dist_idx[1]]} "
              f"(расстояние: {medoid_distances[max_dist_idx]:.4f})")

In [35]:
embeddings_list = umap_embeds.tolist()

if len(embeddings_list) == len(en_featured_df):
    en_featured_df['embeddings'] = embeddings_list
else:
    print("Количество эмбеддингов не совпадает с количеством строк в DataFrame.")

In [ ]:
ensemble = MetaphorClusteringEnsemble(
    data=np.array(en_featured_df[en_featured_df['found_verbs'] == 'kick (пнуть)']['embeddings'].tolist()),
    algorithm_weights={
          'hdbscan': 0.5,  # Наибольший вес для HDBSCAN как самого надежного
          'dbscan': 0.2,
          'spectral': 0.3
      }
)

final_clusters = ensemble.fit(consensus_threshold=0.4)

In [ ]:
visualize_consensus_clusters(
     data=np.array(en_featured_df[en_featured_df['found_verbs'] == 'kick (пнуть)']['embeddings'].tolist()),
     clusters=final_clusters,
     method='both'
)

In [ ]:
detailed_results = analyze_cluster_medoids(
    data=np.array(en_featured_df[en_featured_df['found_verbs'] == 'kick (пнуть)']['embeddings'].tolist()),
    clusters=final_clusters,
    contexts=np.array(en_featured_df[en_featured_df['found_verbs'] == 'kick (пнуть)']['context'].tolist()),
    return_indices=True
)
compare_medoids_similarity(detailed_results, np.array(en_featured_df[en_featured_df['found_verbs'] == 'kick (пнуть)']['embeddings'].tolist()))